In [1]:
import pandas as pd
import numpy as np

# データの読み込み (パスは適宜変更してください)
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# 学習データとテストデータの結合 (欠損値補完用)
all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

print("=== 学習データの基本情報 ===")
print(train_df.info())
print("\n=== テストデータの基本情報 ===")
print(test_df.info())
print("\n=== 全データの欠損値状況 ===")
print(all_data.isnull().sum())

# 欠損値補完の順序を定義 (カテゴリ変数 -> 数値変数, 依存関係を考慮)
# まずは 'HomePlanet' から始めましょう。これが比較的独立してそうなので。
imputation_order = [
    'HomePlanet', # 相対的に独立?
    # 以下、順次追加していきます
    # 'CryoSleep', # VIPや消費金額と関係ありそう?
    # 'Destination', # HomePlanetやCryoSleepと関係あるかも?
    # 'Age', # HomePlanet, CryoSleep, Cabinなどと関係ありそう
    # 'VIP', # 特定のHomePlanetやCryoSleepと関係ありそう?
    # 'Cabin', # HomePlanetやDestinationと関係ありそう (Deck/Side)
    # 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' # 相互に関連、CryoSleep, VIP, Cabinと関係
    # 'Name' # 最後? または別途処理?
]

print("\n=== 欠損値補完順序 (今回の対象: 最初の項目のみ) ===")
print(imputation_order)

# 補完対象の最初のカラム
target_col = imputation_order[0]
print(f"\n--- 補完対象カラム: {target_col} ---")

# 補完対象カラムの情報を表示
print(f"{target_col} のデータ型: {all_data[target_col].dtype}")
print(f"{target_col} のユニーク値: {all_data[target_col].unique()}")
print(f"{target_col} の欠損値数: {all_data[target_col].isnull().sum()}")

# 欠損値補完に使用する特徴量の候補 (カテゴリ変数はOne-Hot, 数値変数はそのまま?)
# PassengerId は GroupID と GroupSize に変換した方が良いかもしれないが、まずは基本的なもので。
# Name は一旦除外。Cabin は Deck/Side に分解予定だが、まずはそのまま?
initial_features = [
    'CryoSleep', 'VIP', 'Destination',
    # 数値変数 (Cabinは文字列なので除く、Nameも除く)
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

print(f"\n--- {target_col} の補完に使用する初期特徴量候補 ---")
print(initial_features)

# データ準備: 特徴量の選別と前処理 (One-Hot Encodingなど)
# 1. 欠損値補完対象でない行を抽出 (ターゲットが存在する行)
non_null_target_df = all_data[all_data[target_col].notnull()].copy()
print(f"\n{target_col} が非欠損のデータ数: {len(non_null_target_df)}")

# 2. 使用する特徴量を抽出
features_df = non_null_target_df[initial_features].copy()

# 3. カテゴリ変数のOne-Hot Encoding (Destination)
features_df = pd.get_dummies(features_df, columns=['Destination'], dummy_na=True) # dummy_na=True で欠損もカテゴリ化

# 4. バイナリ変数の処理 (CryoSleep, VIP)
# 欠損値はFalseとして扱うか、別途補完が必要か？ここでは簡単のため、欠損はFalseとして仮置き
features_df['CryoSleep'] = features_df['CryoSleep'].fillna(False).astype(int)
features_df['VIP'] = features_df['VIP'].fillna(False).astype(int)

# 5. 数値変数の欠損値処理 (一時的に平均値で補完 - モデル学習用)
# 実際の補完モデルの精度を上げるには、ここも別の方法が望ましいが、まずは進めます。
numeric_cols_for_impute_model = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_impute_model:
     if col in features_df.columns: # 存在する場合のみ処理
        features_df[col] = features_df[col].fillna(features_df[col].mean())

# 6. 説明変数 (X) と目的変数 (y) の分離
X = features_df
y = non_null_target_df[target_col]

print(f"\n--- {target_col} 補完モデル用データ準備完了 ---")
print("特徴量 (X) の形状:", X.shape)
print("目的変数 (y) の形状:", y.shape)
print("特徴量カラム:", X.columns.tolist())

# 次のセルでモデルの学習と予測を行います。

=== 学習データの基本情報 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None

=== テストデータの基本情報 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 

/tmp/ipykernel_39046/681675186.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_df['CryoSleep'] = features_df['CryoSleep'].fillna(False).astype(int)
/tmp/ipykernel_39046/681675186.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_df['VIP'] = features_df['VIP'].fillna(False).astype(int)


In [2]:
# --- データ準備 (前セルの続き) ---
# X, y は前セルで定義済みです。
# X.shape = (12682, 12)
# y.shape = (12682,)  # dtype: object, unique: ['Europa' 'Earth' 'Mars']

# --- 1. GPU 利用確認 (今回は省略し、すべて False とします) ---
print("=== GPU 利用確認 ===")
gpu_available = {
    'LightGBM': False,
    'XGBoost': False,
    'CatBoost': False
}
print(f"GPU 利用可否状況 (今回は全て False): {gpu_available}")

# --- 2. モデル定義 (事前設定パラメータ & CPU) ---
print("\n=== モデル定義 (事前設定パラメータ & CPU) ===")
models = {}

# LightGBM (CPU)
import lightgbm as lgb
lgb_params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'max_depth': -1,
    'learning_rate': 0.05,
    'n_estimators': 200,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models['LightGBM'] = lgb.LGBMClassifier(**lgb_params)

# XGBoost (CPU)
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_params = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist')")
# XGBoost 用のモデルは、後で y を変換してから作成・評価します。

# CatBoost (CPU)
import catboost as cb
cb_params = {
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'iterations': 200,
    'depth': 6,
    'learning_rate': 0.05,
    'l2_leaf_reg': 3.0,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models['CatBoost'] = cb.CatBoostClassifier(**cb_params)

# --- 3. モデル評価 (交差検証) ---
print("\n=== モデル評価 (交差検証) ===")
from sklearn.model_selection import StratifiedKFold, cross_val_score
import time

# 交差検証の設定
cv_folds = 5
# XGBoost 用に LabelEncoder を準備
le = LabelEncoder()
y_encoded = le.fit_transform(y) # Earth->0, Europa->1, Mars->2 などに変換
# print(f"元のラベル: {le.classes_}") # 変換前後の対応を確認できます
# print(f"変換後のラベル: {np.unique(y_encoded)}")

skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric = 'accuracy'

model_scores = {}
model_times = {}

# LightGBM と CatBoost の評価 (元のラベル y を使用)
for name in ['LightGBM', 'CatBoost']:
    model = models[name]
    print(f"Evaluating {name}...")
    start_time = time.time()
    try:
        scores = cross_val_score(model, X, y, cv=skf, scoring=scoring_metric, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name}: {e}")
        model_scores[name] = np.nan
        model_times[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores[name] = mean_score
    model_times[name] = elapsed_time
    print(f"  {name} - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# XGBoost の評価 (数値ラベル y_encoded を使用)
print(f"Evaluating XGBoost...")
model_xgb = xgb.XGBClassifier(**xgb_params, enable_categorical=False)
start_time = time.time()
try:
    # XGBoost にはエンコードされた y を渡す
    scores = cross_val_score(model_xgb, X, y_encoded, cv=skf, scoring=scoring_metric, n_jobs=1)
except Exception as e:
    print(f"  Error during CV for XGBoost: {e}")
    model_scores['XGBoost'] = np.nan
    model_times['XGBoost'] = time.time() - start_time
else:
    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores['XGBoost'] = mean_score
    model_times['XGBoost'] = elapsed_time
    print(f"  XGBoost - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 4. 結果出力 ---
print("\n=== モデル比較結果 (CV Accuracy) ===")
sorted_models = sorted(
    [(name, score) for name, score in model_scores.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models:
    print(f"{name}: {score:.5f} (Time: {model_times[name]:.2f}s)")

if sorted_models:
    best_model_name, best_model_score = sorted_models[0]
    print(f"\n--- 最も精度の高いモデル: {best_model_name} (CV Accuracy: {best_model_score:.5f}) ---")
else:
    print("\n--- モデル評価結果がありません ---")
    best_model_name = None

# 次のセルで、選ばれた最良モデルで実際に欠損値を補完します。
# best_model_name に応じて、適切な y (または y_encoded) とモデルを使用して学習・予測

=== GPU 利用確認 ===
GPU 利用可否状況 (今回は全て False): {'LightGBM': False, 'XGBoost': False, 'CatBoost': False}

=== モデル定義 (事前設定パラメータ & CPU) ===
LightGBM: CPU 使用設定
XGBoost: CPU 使用設定 (tree_method='hist')
CatBoost: CPU 使用設定

=== モデル評価 (交差検証) ===
Evaluating LightGBM...
  LightGBM - CV Accuracy: 0.74933 (+/- 0.01270), Time: 14.07s
Evaluating CatBoost...
  CatBoost - CV Accuracy: 0.74420 (+/- 0.00575), Time: -10.15s
Evaluating XGBoost...
  XGBoost - CV Accuracy: 0.74539 (+/- 0.01031), Time: 1.99s

=== モデル比較結果 (CV Accuracy) ===
LightGBM: 0.74933 (Time: 14.07s)
XGBoost: 0.74539 (Time: 1.99s)
CatBoost: 0.74420 (Time: -10.15s)

--- 最も精度の高いモデル: LightGBM (CV Accuracy: 0.74933) ---


In [3]:
# --- 前提条件 ---
# best_model_name = 'LightGBM' (前セルの結果)
# all_data: 元の全データ (train + test 結合)
# X: モデル学習に使った特徴量 (HomePlanet 予測用)
# y: モデル学習に使った目的変数 (HomePlanet, 文字列ラベル)
# initial_features: X を作成したのに使ったカラム名リスト

print("=== HomePlanet 欠損値補完の準備 ===")
# 1. 補完対象データの特定 (HomePlanet が欠損している行)
missing_homeplanet_mask = all_data['HomePlanet'].isnull()
missing_homeplanet_indices = all_data[missing_homeplanet_mask].index
print(f"HomePlanet が欠損しているデータ数: {len(missing_homeplanet_indices)}")

# 2. 補完に使用する特徴量の準備 (欠損行のみ)
#    X (学習データ) と同じ前処理を適用する必要があります。
features_for_prediction_df = all_data.loc[missing_homeplanet_indices, initial_features].copy()

# 3. 特徴量の前処理 (学習時と同様)
#    - Destination の One-Hot Encoding
features_for_prediction_df = pd.get_dummies(features_for_prediction_df, columns=['Destination'], dummy_na=True)

#    - 学習データの One-Hot 後のカラムに合わせる (存在しないカテゴリは0で埋める)
#      例: 学習データに 'Destination_nan' が無く、予測データにあった場合、削除する。
#      例: 学習データに 'Destination_PSO J318.5-22' があり、予測データに無かった場合、0で追加する。
#      `X` のカラムが基準
for col in X.columns: # X は学習に使った特徴量
    if col not in features_for_prediction_df.columns:
        features_for_prediction_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
for col in features_for_prediction_df.columns:
    if col not in X.columns:
        features_for_prediction_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除

#    - カラムの並び順を学習時と一致させる
features_for_prediction_df = features_for_prediction_df[X.columns] # X.columns は学習時のカラム順序

#    - CryoSleep, VIP の処理 (学習時と同様に欠損は False と仮定)
features_for_prediction_df['CryoSleep'] = features_for_prediction_df['CryoSleep'].fillna(False).astype(int)
features_for_prediction_df['VIP'] = features_for_prediction_df['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理 (学習時の平均値で補完 - モデル入力用)
#      重要: 学習データ X の平均値を使う！
numeric_cols_for_impute_model = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_impute_model:
    if col in features_for_prediction_df.columns:
        # 学習データ X の平均値を取得
        train_mean = X[col].mean()
        features_for_prediction_df[col] = features_for_prediction_df[col].fillna(train_mean)

# 4. モデルの学習 (評価済みの best_model_name が 'LightGBM' であることを前提)
print(f"\n=== HomePlanet 補完モデル ({best_model_name}) の学習 ===")
# 評価時に使用した X, y を使って最終モデルを学習
final_imputer_model = models[best_model_name] # models は前セルで定義済み
final_imputer_model.fit(X, y) # LightGBM は文字列ラベル y をそのまま使える
print("学習完了")

# 5. 欠損値の予測
print("\n=== HomePlanet 欠損値の予測 ===")
predicted_homeplanets = final_imputer_model.predict(features_for_prediction_df)
print(f"予測された HomePlanet ラベル: {predicted_homeplanets}")

# 6. 元のデータフレームに補完結果を反映
print("\n=== 補完結果の反映 ===")
all_data.loc[missing_homeplanet_indices, 'HomePlanet'] = predicted_homeplanets
print("HomePlanet 列の欠損値が補完されました。")

# 7. 新しいCSVファイルへの保存 (修正箇所: ディレクトリの存在確認と作成)
import os # os モジュールのインポートを追加

output_file_path = '../data/all_data_imputed_step1_HomePlanet.csv'

# ディレクトリ 'data' が存在しない場合は作成
directory = os.path.dirname(output_file_path)
if directory and not os.path.exists(directory): # directory が空文字でないことを確認
    os.makedirs(directory)
    print(f"ディレクトリ '{directory}' を作成しました。")

all_data.to_csv(output_file_path, index=False)
print(f"\n補完後の全データを '{output_file_path}' に保存しました。")

# 8. 次のステップへの準備
# 次は CryoSleep です。その前に、データの状態を確認します。
print(f"\n=== 次の補完対象カラム 'CryoSleep' の欠損状況 ===")
print(f"'CryoSleep' の欠損値数: {all_data['CryoSleep'].isnull().sum()}")

# 次のステップでは、'CryoSleep' の補完モデルを作成・評価します。
# 必要に応じて、'HomePlanet' の補完結果も特徴量として使用できます。
# 次のセルで進めていきましょう。

=== HomePlanet 欠損値補完の準備 ===
HomePlanet が欠損しているデータ数: 288

=== HomePlanet 補完モデル (LightGBM) の学習 ===


/tmp/ipykernel_39046/3076633070.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_df['CryoSleep'] = features_for_prediction_df['CryoSleep'].fillna(False).astype(int)
/tmp/ipykernel_39046/3076633070.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_df['VIP'] = features_for_prediction_df['VIP'].fillna(False).astype(int)


学習完了

=== HomePlanet 欠損値の予測 ===
予測された HomePlanet ラベル: ['Earth' 'Europa' 'Europa' 'Mars' 'Europa' 'Earth' 'Earth' 'Mars' 'Earth'
 'Europa' 'Mars' 'Earth' 'Earth' 'Europa' 'Mars' 'Europa' 'Earth' 'Mars'
 'Earth' 'Mars' 'Europa' 'Europa' 'Mars' 'Earth' 'Earth' 'Earth' 'Europa'
 'Earth' 'Europa' 'Earth' 'Earth' 'Earth' 'Earth' 'Europa' 'Earth' 'Earth'
 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth'
 'Earth' 'Earth' 'Earth' 'Mars' 'Earth' 'Mars' 'Earth' 'Europa' 'Mars'
 'Earth' 'Earth' 'Earth' 'Europa' 'Earth' 'Earth' 'Mars' 'Earth' 'Earth'
 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth'
 'Earth' 'Europa' 'Earth' 'Earth' 'Earth' 'Europa' 'Earth' 'Europa'
 'Earth' 'Earth' 'Earth' 'Europa' 'Earth' 'Earth' 'Earth' 'Europa' 'Earth'
 'Europa' 'Mars' 'Earth' 'Earth' 'Earth' 'Earth' 'Earth' 'Mars' 'Earth'
 'Europa' 'Europa' 'Earth' 'Europa' 'Earth' 'Earth' 'Earth' 'Mars' 'Mars'
 'Earth' 'Earth' 'Earth' 'Earth' 'Europa' 'Earth' 'Earth' 'Earth' 'Eart

In [4]:
# --- 前提条件 ---
# all_data は HomePlanet が補完された状態

print("=== CryoSleep 欠損値補完の準備 ===")

# 1. 補完対象データの特定 (CryoSleep が欠損している行)
missing_cryosleep_mask = all_data['CryoSleep'].isnull()
missing_cryosleep_indices = all_data[missing_cryosleep_mask].index
print(f"CryoSleep が欠損しているデータ数: {len(missing_cryosleep_indices)}")

# 2. 特徴量の選定
cryosleep_features = [
    'HomePlanet', 'Destination', 'Age', 'VIP',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

print(f"\n--- CryoSleep の補完に使用する特徴量候補 ---")
print(cryosleep_features)

# 3. モデル学習用データの準備 (CryoSleep がわかっているデータ)
non_null_cryosleep_df = all_data[all_data['CryoSleep'].notnull()].copy()
print(f"\nCryoSleep が非欠損のデータ数: {len(non_null_cryosleep_df)}")

#    特徴量と目的変数の分離
X_cryosleep = non_null_cryosleep_df[cryosleep_features].copy()
y_cryosleep = non_null_cryosleep_df['CryoSleep'].copy() # bool型のはず

# --- 重要な追加: y_cryosleep のデータ型を確認・修正 ---
print(f"\n--- y_cryosleep のデータ型確認 ---")
print(f"y_cryosleep の dtype: {y_cryosleep.dtype}")
print(f"y_cryosleep のユニーク値: {y_cryosleep.unique()}")

# データ型が bool でない場合、明示的に変換
# pd.BooleanDtype() を使うと、NaN も扱える nullable boolean になるが、
# scikit-learn モデルには bool が普通使われる。
# ここでは、non-null なデータだけを取り出しているので、標準の bool でOK。
# ただし、念のため object 型なら変換を試みる。
if y_cryosleep.dtype == 'object':
    # object 型の場合、True/False/'True'/'False' などを bool に変換
    y_cryosleep = y_cryosleep.map({'True': True, 'False': False, True: True, False: False})
    print(f"y_cryosleep を object から bool に変換しました。")
    print(f"変換後の dtype: {y_cryosleep.dtype}")
    print(f"変換後のユニーク値: {y_cryosleep.unique()}")
elif y_cryosleep.dtype != 'bool':
    # その他の型の場合も、念のため変換を試みる
    try:
        y_cryosleep = y_cryosleep.astype(bool)
        print(f"y_cryosleep を {y_cryosleep.dtype} から bool に変換しました。")
    except (ValueError, TypeError) as e:
        print(f"y_cryosleep の型変換中にエラーが発生しました: {e}")
        print("モデル評価を続行できません。")
        raise # エラーを再送出して処理を停止

# 4. 特徴量の前処理
print(f"\n--- 特徴量の前処理 ---")
#    - カテゴリ変数の処理 (HomePlanet, Destination)
X_cryosleep_processed = pd.get_dummies(X_cryosleep, columns=['HomePlanet', 'Destination'], dummy_na=True)

#    - バイナリ変数の処理 (VIP)
#      欠損値は False として扱います。
#      警告を避けるために、infer_objects を使うか、astype 前に明示的に処理
X_cryosleep_processed['VIP'] = X_cryosleep_processed['VIP'].fillna(False)
# pd.set_option('future.no_silent_downcasting', True) # 必要に応じて有効化
X_cryosleep_processed['VIP'] = X_cryosleep_processed['VIP'].astype(int) # True/False -> 1/0

#    - 数値変数の欠損値処理
numeric_cols_for_cryosleep = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_cryosleep:
    if col in X_cryosleep_processed.columns:
        X_cryosleep_processed[col] = X_cryosleep_processed[col].fillna(X_cryosleep_processed[col].mean())

print("特徴量前処理完了")

# 5. モデルの定義と比較 (CPU モード)
print("\n=== CryoSleep 補完モデルの定義 (CPU) ===")
# (モデル定義部分は変更なし)
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

models_cryosleep = {}

# LightGBM (CPU)
lgb_params_cs = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models_cryosleep['LightGBM'] = lgb.LGBMClassifier(**lgb_params_cs)

# XGBoost (CPU)
xgb_params_cs = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist')")
models_cryosleep['XGBoost'] = xgb.XGBClassifier(**xgb_params_cs, enable_categorical=False)

# CatBoost (CPU)
cb_params_cs = {
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models_cryosleep['CatBoost'] = cb.CatBoostClassifier(**cb_params_cs)

# RandomForest
rf_params_cs = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest: 設定")
models_cryosleep['RandomForest'] = RandomForestClassifier(**rf_params_cs)

# --- 6. モデル評価 (交差検証) ---
print("\n=== CryoSleep モデル評価 (交差検証) ===")
from sklearn.model_selection import StratifiedKFold, cross_val_score
import time

# 交差検証の設定
cv_folds = 5
skf_cs = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric_cs = 'accuracy'

model_scores_cs = {}
model_times_cs = {}

for name, model in models_cryosleep.items():
    print(f"Evaluating {name}...")
    start_time = time.time()
    try:
        # n_jobs=1 にして、並列処理によるトラブルを避ける
        scores = cross_val_score(model, X_cryosleep_processed, y_cryosleep, cv=skf_cs, scoring=scoring_metric_cs, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name}: {e}")
        model_scores_cs[name] = np.nan
        model_times_cs[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_cs[name] = mean_score
    model_times_cs[name] = elapsed_time

    print(f"  {name} - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 7. 結果出力 ---
print("\n=== CryoSleep モデル比較結果 (CV Accuracy) ===")
sorted_models_cs = sorted(
    [(name, score) for name, score in model_scores_cs.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models_cs:
    print(f"{name}: {score:.5f} (Time: {model_times_cs[name]:.2f}s)")

if sorted_models_cs:
    best_model_name_cs, best_model_score_cs = sorted_models_cs[0]
    print(f"\n--- CryoSleep 補完に最も適したモデル: {best_model_name_cs} (CV Accuracy: {best_model_score_cs:.5f}) ---")
else:
    print("\n--- CryoSleep モデル評価結果がありません ---")
    best_model_name_cs = None

# 次のセルで、選ばれた最良モデルで実際に CryoSleep の欠損値を補完します。

=== CryoSleep 欠損値補完の準備 ===
CryoSleep が欠損しているデータ数: 310

--- CryoSleep の補完に使用する特徴量候補 ---
['HomePlanet', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

CryoSleep が非欠損のデータ数: 12660

--- y_cryosleep のデータ型確認 ---
y_cryosleep の dtype: object
y_cryosleep のユニーク値: [False True]
y_cryosleep を object から bool に変換しました。
変換後の dtype: bool
変換後のユニーク値: [False  True]

--- 特徴量の前処理 ---
特徴量前処理完了

=== CryoSleep 補完モデルの定義 (CPU) ===
LightGBM: CPU 使用設定
XGBoost: CPU 使用設定 (tree_method='hist')
CatBoost: CPU 使用設定
RandomForest: 設定

=== CryoSleep モデル評価 (交差検証) ===
Evaluating LightGBM...


/tmp/ipykernel_39046/2446711619.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_cryosleep_processed['VIP'] = X_cryosleep_processed['VIP'].fillna(False)


  LightGBM - CV Accuracy: 0.95340 (+/- 0.00522), Time: 12.66s
Evaluating XGBoost...
  XGBoost - CV Accuracy: 0.95427 (+/- 0.00437), Time: -11.79s
Evaluating CatBoost...
  CatBoost - CV Accuracy: 0.95016 (+/- 0.00627), Time: 0.96s
Evaluating RandomForest...
  RandomForest - CV Accuracy: 0.93160 (+/- 0.00726), Time: 0.94s

=== CryoSleep モデル比較結果 (CV Accuracy) ===
XGBoost: 0.95427 (Time: -11.79s)
LightGBM: 0.95340 (Time: 12.66s)
CatBoost: 0.95016 (Time: 0.96s)
RandomForest: 0.93160 (Time: 0.94s)

--- CryoSleep 補完に最も適したモデル: XGBoost (CV Accuracy: 0.95427) ---


In [5]:
# --- 前提条件 ---
# best_model_name_cs = 'XGBoost' (前セルの結果)
# all_data は HomePlanet が補完された状態 (メモリ上または ../data/all_data_imputed_step1_HomePlanet.csv から読み込み済み)
# X_cryosleep_processed, y_cryosleep は前セルで定義済み (モデル学習用の特徴量と目的変数)
# cryosleep_features は前セルで定義済み (使用する特徴量のリスト)
# models_cryosleep は前セルで定義済み (モデル辞書)

print("=== CryoSleep 欠損値補完の実行 ===")

# 1. 補完対象データの特定 (CryoSleep が欠損している行)
missing_cryosleep_mask = all_data['CryoSleep'].isnull()
missing_cryosleep_indices = all_data[missing_cryosleep_mask].index
print(f"CryoSleep が欠損しているデータ数: {len(missing_cryosleep_indices)}")

# 2. 補完に使用する特徴量の準備 (欠損行のみ)
features_for_prediction_cs_df = all_data.loc[missing_cryosleep_indices, cryosleep_features].copy()

# 3. 特徴量の前処理 (学習時と同様)
#    - カテゴリ変数の処理 (HomePlanet, Destination)
features_for_prediction_cs_df = pd.get_dummies(features_for_prediction_cs_df, columns=['HomePlanet', 'Destination'], dummy_na=True)

#    - 学習データの One-Hot 後のカラムに合わせる
#      `X_cryosleep_processed` のカラムが基準
for col in X_cryosleep_processed.columns: # 学習時のカラム
    if col not in features_for_prediction_cs_df.columns:
        features_for_prediction_cs_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
for col in features_for_prediction_cs_df.columns:
    if col not in X_cryosleep_processed.columns:
        features_for_prediction_cs_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除

#    - カラムの並び順を学習時と一致させる
features_for_prediction_cs_df = features_for_prediction_cs_df[X_cryosleep_processed.columns]

#    - バイナリ変数の処理 (VIP)
features_for_prediction_cs_df['VIP'] = features_for_prediction_cs_df['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理 (学習データの平均値で補完 - モデル入力用)
#      重要: 学習データ X_cryosleep_processed の平均値を使う！
numeric_cols_for_cryosleep = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_cryosleep:
    if col in features_for_prediction_cs_df.columns:
        # 学習データの平均値を取得
        train_mean = X_cryosleep_processed[col].mean()
        features_for_prediction_cs_df[col] = features_for_prediction_cs_df[col].fillna(train_mean)

# 4. モデルの学習 (評価済みの best_model_name_cs が 'XGBoost' であることを前提)
print(f"\n=== CryoSleep 補完モデル ({best_model_name_cs}) の学習 ===")
# 評価時に使用した X_cryosleep_processed, y_cryosleep を使って最終モデルを学習
final_imputer_model_cs = models_cryosleep[best_model_name_cs] # models_cryosleep は前セルで定義済み
final_imputer_model_cs.fit(X_cryosleep_processed, y_cryosleep) # XGBoost は bool ラベル y_cryosleep をそのまま使える
print("学習完了")

# 5. 欠損値の予測
print("\n=== CryoSleep 欠損値の予測 ===")
predicted_cryosleeps = final_imputer_model_cs.predict(features_for_prediction_cs_df)
# 予測結果は bool 型 (True/False) であることを確認
print(f"予測された CryoSleep ラベル (最初の10件): {predicted_cryosleeps[:10]}")
print(f"予測結果のデータ型: {predicted_cryosleeps.dtype}")

# 6. 元のデータフレームに補完結果を反映
print("\n=== 補完結果の反映 ===")
# bool 型の予測結果を直接代入
all_data.loc[missing_cryosleep_indices, 'CryoSleep'] = predicted_cryosleeps
print("CryoSleep 列の欠損値が補完されました。")

# 7. 新しいCSVファイルへの保存
# ファイルパスの確認と作成 (必要に応じて)
import os
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"ディレクトリ '{data_dir}' を作成しました。")

output_file_path_cs = os.path.join(data_dir, 'all_data_imputed_step2_CryoSleep.csv')
all_data.to_csv(output_file_path_cs, index=False)
print(f"\n補完後の全データを '{output_file_path_cs}' に保存しました。")

# 8. 次のステップへの準備
# 次は Destination です。その前に、データの状態を確認します。
print(f"\n=== 次の補完対象カラム 'Destination' の欠損状況 ===")
print(f"'Destination' の欠損値数: {all_data['Destination'].isnull().sum()}")

# 次のステップでは、'Destination' の補完モデルを作成・評価します。
# CryoSleep の補完結果も特徴量として使用できます。
# 次のセル (セル 6) で進めていきましょう。

=== CryoSleep 欠損値補完の実行 ===
CryoSleep が欠損しているデータ数: 310

=== CryoSleep 補完モデル (XGBoost) の学習 ===


/tmp/ipykernel_39046/1675162488.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_cs_df['VIP'] = features_for_prediction_cs_df['VIP'].fillna(False).astype(int)


学習完了

=== CryoSleep 欠損値の予測 ===
予測された CryoSleep ラベル (最初の10件): [1 0 0 1 0 1 0 1 0 1]
予測結果のデータ型: int64

=== 補完結果の反映 ===
CryoSleep 列の欠損値が補完されました。

補完後の全データを '../data/all_data_imputed_step2_CryoSleep.csv' に保存しました。

=== 次の補完対象カラム 'Destination' の欠損状況 ===
'Destination' の欠損値数: 274


In [6]:
# --- 前提条件 ---
# all_data は HomePlanet と CryoSleep が補完された状態
# ここでは all_data が既に更新されている前提

print("=== Destination 欠損値補完の準備 ===")

# 1. 補完対象データの特定 (Destination が欠損している行)
missing_destination_mask = all_data['Destination'].isnull()
missing_destination_indices = all_data[missing_destination_mask].index
print(f"Destination が欠損しているデータ数: {len(missing_destination_indices)}")

# 2. 特徴量の選定
destination_features = [
    'HomePlanet', 'CryoSleep', 'Age', 'VIP',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

print(f"\n--- Destination の補完に使用する特徴量候補 ---")
print(destination_features)

# 3. モデル学習用データの準備 (Destination がわかっているデータ)
non_null_destination_df = all_data[all_data['Destination'].notnull()].copy()
print(f"\nDestination が非欠損のデータ数: {len(non_null_destination_df)}")

#    特徴量と目的変数の分離
X_destination = non_null_destination_df[destination_features].copy()
y_destination = non_null_destination_df['Destination'].copy() # object 型 (カテゴリ)

# 4. 特徴量の前処理
print(f"\n--- 特徴量の前処理 ---")
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep)
X_destination_processed = pd.get_dummies(X_destination, columns=['HomePlanet', 'CryoSleep'], dummy_na=True)

#    - バイナリ変数の処理 (VIP)
X_destination_processed['VIP'] = X_destination_processed['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理
numeric_cols_for_destination = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_destination:
    if col in X_destination_processed.columns:
        X_destination_processed[col] = X_destination_processed[col].fillna(X_destination_processed[col].mean())

# 5. モデルの定義と比較 (CPU モード)
print("\n=== Destination 補完モデルの定義 (CPU) ===")
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.preprocessing import LabelEncoder

models_destination = {}

# LightGBM (CPU)
lgb_params_dest = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models_destination['LightGBM'] = lgb.LGBMClassifier(**lgb_params_dest)

# XGBoost (CPU) - 修正箇所: LabelEncoder を使用
xgb_params_dest = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist') - LabelEncoder を使用")
# XGBoost 用のモデルは、後で y_destination を変換してから作成・評価します。

# CatBoost (CPU)
cb_params_dest = {
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models_destination['CatBoost'] = cb.CatBoostClassifier(**cb_params_dest)

# RandomForest
rf_params_dest = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest: 設定")
models_destination['RandomForest'] = RandomForestClassifier(**rf_params_dest)

# --- 6. モデル評価 (交差検証) ---
print("\n=== Destination モデル評価 (交差検証) ===")
from sklearn.model_selection import StratifiedKFold, cross_val_score
import time

# 交差検証の設定
cv_folds = 5
skf_dest = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric_dest = 'accuracy'

# XGBoost 用に LabelEncoder を準備
le_dest = LabelEncoder()
y_destination_encoded = le_dest.fit_transform(y_destination) # 文字列 -> 0, 1, 2

model_scores_dest = {}
model_times_dest = {}

# LightGBM, CatBoost, RandomForest の評価 (元のラベル y_destination を使用)
for name in ['LightGBM', 'CatBoost', 'RandomForest']:
    model = models_destination[name]
    print(f"Evaluating {name}...")
    start_time = time.time()
    try:
        scores = cross_val_score(model, X_destination_processed, y_destination, cv=skf_dest, scoring=scoring_metric_dest, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name}: {e}")
        model_scores_dest[name] = np.nan
        model_times_dest[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_dest[name] = mean_score
    model_times_dest[name] = elapsed_time
    print(f"  {name} - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# XGBoost の評価 (数値ラベル y_destination_encoded を使用)
print(f"Evaluating XGBoost...")
model_xgb_dest = xgb.XGBClassifier(**xgb_params_dest, enable_categorical=False)
start_time = time.time()
try:
    scores = cross_val_score(model_xgb_dest, X_destination_processed, y_destination_encoded, cv=skf_dest, scoring=scoring_metric_dest, n_jobs=1)
except Exception as e:
    print(f"  Error during CV for XGBoost: {e}")
    model_scores_dest['XGBoost'] = np.nan
    model_times_dest['XGBoost'] = time.time() - start_time
else:
    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_dest['XGBoost'] = mean_score
    model_times_dest['XGBoost'] = elapsed_time
    print(f"  XGBoost - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 7. 結果出力 ---
print("\n=== Destination モデル比較結果 (CV Accuracy) ===")
sorted_models_dest = sorted(
    [(name, score) for name, score in model_scores_dest.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models_dest:
    print(f"{name}: {score:.5f} (Time: {model_times_dest[name]:.2f}s)")

if sorted_models_dest:
    best_model_name_dest, best_model_score_dest = sorted_models_dest[0]
    print(f"\n--- Destination 補完に最も適したモデル: {best_model_name_dest} (CV Accuracy: {best_model_score_dest:.5f}) ---")
else:
    print("\n--- Destination モデル評価結果がありません ---")
    best_model_name_dest = None

# 次のセルで、選ばれた最良モデルで実際に Destination の欠損値を補完します。
# (セル 7 の内容)

=== Destination 欠損値補完の準備 ===
Destination が欠損しているデータ数: 274

--- Destination の補完に使用する特徴量候補 ---
['HomePlanet', 'CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

Destination が非欠損のデータ数: 12696

--- 特徴量の前処理 ---

=== Destination 補完モデルの定義 (CPU) ===
LightGBM: CPU 使用設定
XGBoost: CPU 使用設定 (tree_method='hist') - LabelEncoder を使用
CatBoost: CPU 使用設定
RandomForest: 設定

=== Destination モデル評価 (交差検証) ===
Evaluating LightGBM...


/tmp/ipykernel_39046/1441826967.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_destination_processed['VIP'] = X_destination_processed['VIP'].fillna(False).astype(int)


  LightGBM - CV Accuracy: 0.69274 (+/- 0.01196), Time: 0.88s
Evaluating CatBoost...
  CatBoost - CV Accuracy: 0.69841 (+/- 0.00400), Time: 1.69s
Evaluating RandomForest...
  RandomForest - CV Accuracy: 0.69542 (+/- 0.00997), Time: 0.95s
Evaluating XGBoost...
  XGBoost - CV Accuracy: 0.69455 (+/- 0.01144), Time: 2.34s

=== Destination モデル比較結果 (CV Accuracy) ===
CatBoost: 0.69841 (Time: 1.69s)
RandomForest: 0.69542 (Time: 0.95s)
XGBoost: 0.69455 (Time: 2.34s)
LightGBM: 0.69274 (Time: 0.88s)

--- Destination 補完に最も適したモデル: CatBoost (CV Accuracy: 0.69841) ---


In [7]:
# --- 前提条件 ---
# best_model_name_dest = 'CatBoost' (前セルの結果)
# all_data は HomePlanet と CryoSleep が補完された状態
# X_destination_processed, y_destination は前セルで定義済み (モデル学習用の特徴量と目的変数)
# destination_features は前セルで定義済み (使用する特徴量のリスト)
# models_destination は前セルで定義済み (モデル辞書)
# le_dest は前セルで定義済み (LabelEncoder, XGBoost用だが今回は使わない)

print("=== Destination 欠損値補完の実行 ===")

# 1. 補完対象データの特定 (Destination が欠損している行)
missing_destination_mask = all_data['Destination'].isnull()
missing_destination_indices = all_data[missing_destination_mask].index
print(f"Destination が欠損しているデータ数: {len(missing_destination_indices)}")

# 2. 補完に使用する特徴量の準備 (欠損行のみ)
features_for_prediction_dest_df = all_data.loc[missing_destination_mask, destination_features].copy()

# 3. 特徴量の前処理 (学習時と同様)
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep)
features_for_prediction_dest_df = pd.get_dummies(features_for_prediction_dest_df, columns=['HomePlanet', 'CryoSleep'], dummy_na=True)

#    - 学習データの One-Hot 後のカラムに合わせる
#      `X_destination_processed` のカラムが基準
for col in X_destination_processed.columns: # 学習時のカラム
    if col not in features_for_prediction_dest_df.columns:
        features_for_prediction_dest_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
for col in features_for_prediction_dest_df.columns:
    if col not in X_destination_processed.columns:
        features_for_prediction_dest_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除

#    - カラムの並び順を学習時と一致させる
features_for_prediction_dest_df = features_for_prediction_dest_df[X_destination_processed.columns]

#    - バイナリ変数の処理 (VIP)
features_for_prediction_dest_df['VIP'] = features_for_prediction_dest_df['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理 (学習データの平均値で補完 - モデル入力用)
#      重要: 学習データ X_destination_processed の平均値を使う！
numeric_cols_for_destination = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_destination:
    if col in features_for_prediction_dest_df.columns:
        # 学習データの平均値を取得
        train_mean = X_destination_processed[col].mean()
        features_for_prediction_dest_df[col] = features_for_prediction_dest_df[col].fillna(train_mean)

# 4. モデルの学習 (評価済みの best_model_name_dest が 'CatBoost' であることを前提)
print(f"\n=== Destination 補完モデル ({best_model_name_dest}) の学習 ===")
# 評価時に使用した X_destination_processed, y_destination を使って最終モデルを学習
final_imputer_model_dest = models_destination[best_model_name_dest] # models_destination は前セルで定義済み
final_imputer_model_dest.fit(X_destination_processed, y_destination) # CatBoost は文字列ラベル y_destination をそのまま使える
print("学習完了")

# 5. 欠損値の予測
print("\n=== Destination 欠損値の予測 ===")
predicted_destinations = final_imputer_model_dest.predict(features_for_prediction_dest_df)
# 予測結果は文字列ラベル (例: 'TRAPPIST-1e') であることを確認
print(f"予測された Destination ラベル (最初の10件): {predicted_destinations[:10]}")
print(f"予測結果のデータ型: {type(predicted_destinations[0])}") # 各要素の型を確認

# 6. 元のデータフレームに補完結果を反映
print("\n=== 補完結果の反映 ===")
# 予測結果 (文字列配列) を直接代入
all_data.loc[missing_destination_indices, 'Destination'] = predicted_destinations
print("Destination 列の欠損値が補完されました。")

# 7. 新しいCSVファイルへの保存
# ファイルパスの確認と作成 (必要に応じて)
import os
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"ディレクトリ '{data_dir}' を作成しました。")

output_file_path_dest = os.path.join(data_dir, 'all_data_imputed_step3_Destination.csv')
all_data.to_csv(output_file_path_dest, index=False)
print(f"\n補完後の全データを '{output_file_path_dest}' に保存しました。")

# 8. 次のステップへの準備
# 次は Age です。その前に、データの状態を確認します。
print(f"\n=== 次の補完対象カラム 'Age' の欠損状況 ===")
print(f"'Age' の欠損値数: {all_data['Age'].isnull().sum()}")

# 次のステップでは、'Age' の補完モデルを作成・評価します。
# Destination の補完結果も特徴量として使用できます。
# 次のセル (セル 8) で進めていきましょう。

=== Destination 欠損値補完の実行 ===
Destination が欠損しているデータ数: 274

=== Destination 補完モデル (CatBoost) の学習 ===


/tmp/ipykernel_39046/207029085.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_dest_df['VIP'] = features_for_prediction_dest_df['VIP'].fillna(False).astype(int)


学習完了

=== Destination 欠損値の予測 ===
予測された Destination ラベル (最初の10件): [['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']
 ['TRAPPIST-1e']]
予測結果のデータ型: <class 'numpy.ndarray'>

=== 補完結果の反映 ===
Destination 列の欠損値が補完されました。

補完後の全データを '../data/all_data_imputed_step3_Destination.csv' に保存しました。

=== 次の補完対象カラム 'Age' の欠損状況 ===
'Age' の欠損値数: 270


In [8]:
# --- 前提条件 ---
# all_data は HomePlanet, CryoSleep, Destination が補完された状態 (../data/all_data_imputed_step3_Destination.csv から読み込み済み or メモリ上)
# ここでは all_data が既に更新されている前提

print("=== Age 欠損値補完の準備 ===")

# 1. 補完対象データの特定 (Age が欠損している行)
missing_age_mask = all_data['Age'].isnull()
missing_age_indices = all_data[missing_age_mask].index
print(f"Age が欠損しているデータ数: {len(missing_age_indices)}")

# 2. 特徴量の選定 (Age 予測に使えそうなもの)
#    これまで補完した HomePlanet, CryoSleep, Destination も特徴量に含めます。
#    Cabin は Deck/Side 分解が有効かもしれませんが、まずは文字列全体 or 分解版 (後で追加も可)。
#    Name, PassengerId は一旦除外します。
#    消費金額も特徴量に含めます。
age_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
    # 'Cabin' # 後で Deck/Side に分解して追加検討
]

print(f"\n--- Age の補完に使用する特徴量候補 ---")
print(age_features)

# 3. モデル学習用データの準備 (Age がわかっているデータ)
non_null_age_df = all_data[all_data['Age'].notnull()].copy()
print(f"\nAge が非欠損のデータ数: {len(non_null_age_df)}")

#    特徴量と目的変数の分離
X_age = non_null_age_df[age_features].copy()
y_age = non_null_age_df['Age'].copy() # float64 型 (数値)

# 4. 特徴量の前処理
print(f"\n--- 特徴量の前処理 ---")
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination)
#      One-Hot Encoding を使用します。
X_age_processed = pd.get_dummies(X_age, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

#    - バイナリ変数の処理 (VIP)
#      欠損値は False として扱います。
X_age_processed['VIP'] = X_age_processed['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理 (RoomService, FoodCourt, ShoppingMall, Spa, VRDeck)
#      学習データの平均値で補完します。
numeric_cols_for_age = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_age:
    if col in X_age_processed.columns: # 存在する場合のみ処理
        X_age_processed[col] = X_age_processed[col].fillna(X_age_processed[col].mean())

# 5. モデルの定義と比較 (CPU モード)
#    Age は数値 (回帰) です。
print("\n=== Age 補完モデルの定義 (CPU) ===")
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import mean_squared_error # 回帰の評価指標

models_age = {}

# LightGBM Regressor (CPU)
lgb_params_age = {
    'objective': 'regression',
    'metric': 'rmse', # 回帰の評価指標
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM Regressor: CPU 使用設定")
models_age['LightGBM'] = lgb.LGBMRegressor(**lgb_params_age)

# XGBoost Regressor (CPU)
xgb_params_age = {
    'objective': 'reg:squarederror', # 回帰 (二乗誤差)
    'eval_metric': 'rmse', # 内部評価指標
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost Regressor: CPU 使用設定 (tree_method='hist')")
models_age['XGBoost'] = xgb.XGBRegressor(**xgb_params_age, enable_categorical=False) # OneHot済みなのでFalse

# CatBoost Regressor (CPU)
cb_params_age = {
    'loss_function': 'RMSE', # 回帰の損失関数
    'eval_metric': 'RMSE',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost Regressor: CPU 使用設定")
models_age['CatBoost'] = cb.CatBoostRegressor(**cb_params_age)

# RandomForest Regressor
rf_params_age = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest Regressor: 設定")
models_age['RandomForest'] = RandomForestRegressor(**rf_params_age)

# --- 6. モデル評価 (交差検証) ---
print("\n=== Age モデル評価 (交差検証) ===")
from sklearn.model_selection import KFold, cross_val_score # 回帰なので KFold
import time

# 交差検証の設定 (回帰用)
cv_folds = 5
kf_age = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
# 回帰の評価指標: RMSE (Root Mean Squared Error) の符号を反転させたもの (neg_root_mean_squared_error)
# sklearn に neg_root_mean_squared_error がない場合、neg_mean_squared_error の平方根を取る
scoring_metric_age = 'neg_mean_squared_error' # または 'r2' (決定係数) も選択可

model_scores_age = {}
model_times_age = {}

for name, model in models_age.items():
    print(f"Evaluating {name}...")
    start_time = time.time()
    try:
        # n_jobs=1 にして、並列処理によるトラブルを避ける
        scores = cross_val_score(model, X_age_processed, y_age, cv=kf_age, scoring=scoring_metric_age, n_jobs=1)
        # scores は負のMSE。RMSEにするには sqrt(-score) が必要。
        # ただし、cross_val_score の結果は平均と標準偏差の比較にそのまま使える。
        rmse_scores = np.sqrt(-scores)
    except Exception as e:
        print(f"  Error during CV for {name}: {e}")
        # エラーが発生した場合はスコアを None とする
        model_scores_age[name] = np.nan
        model_times_age[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_rmse = rmse_scores.mean()
    std_rmse = rmse_scores.std()
    elapsed_time = end_time - start_time

    model_scores_age[name] = mean_rmse # RMSEの平均を格納
    model_times_age[name] = elapsed_time

    print(f"  {name} - CV RMSE: {mean_rmse:.2f} (+/- {std_rmse * 2:.2f}), Time: {elapsed_time:.2f}s")

# --- 7. 結果出力 ---
print("\n=== Age モデル比較結果 (CV RMSE - 小さいほど良い) ===")
# NaN を除外してソート (RMSEが小さい方が良いので reverse=False)
sorted_models_age = sorted(
    [(name, score) for name, score in model_scores_age.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=False # RMSE が小さいほど良い
)
for name, score in sorted_models_age:
    print(f"{name}: RMSE={score:.2f} (Time: {model_times_age[name]:.2f}s)")

if sorted_models_age:
    best_model_name_age, best_model_score_age = sorted_models_age[0]
    print(f"\n--- Age 補完に最も適したモデル: {best_model_name_age} (CV RMSE: {best_model_score_age:.2f}) ---")
else:
    print("\n--- Age モデル評価結果がありません ---")
    best_model_name_age = None

# 次のセルで、選ばれた最良モデルで実際に Age の欠損値を補完します。
# (セル 9 の内容をここに書くか、次のセルに分割するかはお任せします)

=== Age 欠損値補完の準備 ===
Age が欠損しているデータ数: 270

--- Age の補完に使用する特徴量候補 ---
['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

Age が非欠損のデータ数: 12700

--- 特徴量の前処理 ---

=== Age 補完モデルの定義 (CPU) ===
LightGBM Regressor: CPU 使用設定
XGBoost Regressor: CPU 使用設定 (tree_method='hist')
CatBoost Regressor: CPU 使用設定
RandomForest Regressor: 設定

=== Age モデル評価 (交差検証) ===
Evaluating LightGBM...


/tmp/ipykernel_39046/2300592329.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_age_processed['VIP'] = X_age_processed['VIP'].fillna(False).astype(int)


  LightGBM - CV RMSE: 13.09 (+/- 0.38), Time: 0.35s
Evaluating XGBoost...
  XGBoost - CV RMSE: 13.14 (+/- 0.37), Time: 12.90s
Evaluating CatBoost...
  CatBoost - CV RMSE: 13.10 (+/- 0.36), Time: -10.95s
Evaluating RandomForest...
  RandomForest - CV RMSE: 13.11 (+/- 0.37), Time: 0.85s

=== Age モデル比較結果 (CV RMSE - 小さいほど良い) ===
LightGBM: RMSE=13.09 (Time: 0.35s)
CatBoost: RMSE=13.10 (Time: -10.95s)
RandomForest: RMSE=13.11 (Time: 0.85s)
XGBoost: RMSE=13.14 (Time: 12.90s)

--- Age 補完に最も適したモデル: LightGBM (CV RMSE: 13.09) ---


In [9]:
# --- 前提条件 ---
# best_model_name_age = 'LightGBM' (前セルの結果)
# all_data は HomePlanet, CryoSleep, Destination が補完された状態
# X_age_processed, y_age は前セルで定義済み (モデル学習用の特徴量と目的変数)
# age_features は前セルで定義済み (使用する特徴量のリスト)
# models_age は前セルで定義済み (モデル辞書)

print("=== Age 欠損値補完の実行 ===")

# 1. 補完対象データの特定 (Age が欠損している行)
missing_age_mask = all_data['Age'].isnull()
missing_age_indices = all_data[missing_age_mask].index
print(f"Age が欠損しているデータ数: {len(missing_age_indices)}")

# 2. 補完に使用する特徴量の準備 (欠損行のみ)
features_for_prediction_age_df = all_data.loc[missing_age_mask, age_features].copy()

# 3. 特徴量の前処理 (学習時と同様)
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination)
features_for_prediction_age_df = pd.get_dummies(features_for_prediction_age_df, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

#    - 学習データの One-Hot 後のカラムに合わせる
#      `X_age_processed` のカラムが基準
for col in X_age_processed.columns: # 学習時のカラム
    if col not in features_for_prediction_age_df.columns:
        features_for_prediction_age_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
for col in features_for_prediction_age_df.columns:
    if col not in X_age_processed.columns:
        features_for_prediction_age_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除

#    - カラムの並び順を学習時と一致させる
features_for_prediction_age_df = features_for_prediction_age_df[X_age_processed.columns]

#    - バイナリ変数の処理 (VIP)
features_for_prediction_age_df['VIP'] = features_for_prediction_age_df['VIP'].fillna(False).astype(int)

#    - 数値変数の欠損値処理 (学習データの平均値で補完 - モデル入力用)
#      重要: 学習データ X_age_processed の平均値を使う！
numeric_cols_for_age = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_age:
    if col in features_for_prediction_age_df.columns:
        # 学習データの平均値を取得
        train_mean = X_age_processed[col].mean()
        features_for_prediction_age_df[col] = features_for_prediction_age_df[col].fillna(train_mean)

# 4. モデルの学習 (評価済みの best_model_name_age が 'LightGBM' であることを前提)
print(f"\n=== Age 補完モデル ({best_model_name_age}) の学習 ===")
# 評価時に使用した X_age_processed, y_age を使って最終モデルを学習
final_imputer_model_age = models_age[best_model_name_age] # models_age は前セルで定義済み
final_imputer_model_age.fit(X_age_processed, y_age) # LightGBM Regressor は数値 y_age をそのまま使える
print("学習完了")

# 5. 欠損値の予測
print("\n=== Age 欠損値の予測 ===")
predicted_ages = final_imputer_model_age.predict(features_for_prediction_age_df)
# 予測結果は数値 (float) の配列であることを確認
print(f"予測された Age (最初の10件): {predicted_ages[:10]}")
print(f"予測結果のデータ型: {predicted_ages.dtype}")

# 6. 元のデータフレームに補完結果を反映
print("\n=== 補完結果の反映 ===")
# 予測結果 (数値配列) を直接代入
all_data.loc[missing_age_indices, 'Age'] = predicted_ages
print("Age 列の欠損値が補完されました。")

# 7. 新しいCSVファイルへの保存
# ファイルパスの確認と作成 (必要に応じて)
import os
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"ディレクトリ '{data_dir}' を作成しました。")

output_file_path_age = os.path.join(data_dir, 'all_data_imputed_step4_Age.csv')
all_data.to_csv(output_file_path_age, index=False)
print(f"\n補完後の全データを '{output_file_path_age}' に保存しました。")

# 8. 次のステップへの準備
# 次は VIP です。その前に、データの状態を確認します。
print(f"\n=== 次の補完対象カラム 'VIP' の欠損状況 ===")
print(f"'VIP' の欠損値数: {all_data['VIP'].isnull().sum()}")

# 次のステップでは、'VIP' の補完モデルを作成・評価します。
# Age の補完結果も特徴量として使用できます。
# 次のセル (セル 10) で進めていきましょう。

=== Age 欠損値補完の実行 ===
Age が欠損しているデータ数: 270

=== Age 補完モデル (LightGBM) の学習 ===
学習完了

=== Age 欠損値の予測 ===
予測された Age (最初の10件): [29.15730817 31.50955197 21.12326613 35.52049528 38.17648676 32.54892812
 21.69404638 25.94347926 37.13202953 28.1504719 ]
予測結果のデータ型: float64

=== 補完結果の反映 ===
Age 列の欠損値が補完されました。

補完後の全データを '../data/all_data_imputed_step4_Age.csv' に保存しました。

=== 次の補完対象カラム 'VIP' の欠損状況 ===
'VIP' の欠損値数: 296


/tmp/ipykernel_39046/1547524564.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_age_df['VIP'] = features_for_prediction_age_df['VIP'].fillna(False).astype(int)


In [10]:
# --- 前提条件 ---
# all_data は HomePlanet, CryoSleep, Destination, Age が補完された状態 (../data/all_data_imputed_step4_Age.csv から読み込み済み or メモリ上)
# ここでは all_data が既に更新されている前提

print("=== VIP 欠損値補完の準備 ===")

# 1. 補完対象データの特定 (VIP が欠損している行)
missing_vip_mask = all_data['VIP'].isnull()
missing_vip_indices = all_data[missing_vip_mask].index
print(f"VIP が欠損しているデータ数: {len(missing_vip_indices)}")

# 2. 特徴量の選定 (VIP 予測に使えそうなもの)
#    これまで補完した HomePlanet, CryoSleep, Destination, Age も特徴量に含めます。
#    Cabin は Deck/Side 分解が有効かもしれませんが、まずは文字列全体 or 分解版 (後で追加も可)。
#    Name, PassengerId は一旦除外します。
#    消費金額も特徴量に含めます。
vip_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
    # 'Cabin' # 後で Deck/Side に分解して追加検討
]

print(f"\n--- VIP の補完に使用する特徴量候補 ---")
print(vip_features)

# 3. モデル学習用データの準備 (VIP がわかっているデータ)
non_null_vip_df = all_data[all_data['VIP'].notnull()].copy()
print(f"\nVIP が非欠損のデータ数: {len(non_null_vip_df)}")

#    特徴量と目的変数の分離
X_vip = non_null_vip_df[vip_features].copy()
y_vip = non_null_vip_df['VIP'].copy() # object 型 (bool または 'True'/'False' 文字列)

# --- 重要な追加: y_vip のデータ型を確認・修正 ---
print(f"\n--- y_vip のデータ型確認 ---")
print(f"y_vip の dtype: {y_vip.dtype}")
print(f"y_vip のユニーク値: {y_vip.unique()}")

# データ型が bool でない場合、明示的に変換
if y_vip.dtype == 'object':
    # object 型の場合、True/False/'True'/'False' などを bool に変換
    y_vip = y_vip.map({'True': True, 'False': False, True: True, False: False})
    print(f"y_vip を object から bool に変換しました。")
    print(f"変換後の dtype: {y_vip.dtype}")
    print(f"変換後のユニーク値: {y_vip.unique()}")
elif y_vip.dtype != 'bool':
    # その他の型の場合も、念のため変換を試みる
    try:
        y_vip = y_vip.astype(bool)
        print(f"y_vip を {y_vip.dtype} から bool に変換しました。")
    except (ValueError, TypeError) as e:
        print(f"y_vip の型変換中にエラーが発生しました: {e}")
        print("モデル評価を続行できません。")
        raise # エラーを再送出して処理を停止

# 4. 特徴量の前処理
print(f"\n--- 特徴量の前処理 ---")
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination)
#      One-Hot Encoding を使用します。
X_vip_processed = pd.get_dummies(X_vip, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

#    - 数値変数の欠損値処理 (Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck)
#      学習データの平均値で補完します。
numeric_cols_for_vip = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_vip:
    if col in X_vip_processed.columns: # 存在する場合のみ処理
        X_vip_processed[col] = X_vip_processed[col].fillna(X_vip_processed[col].mean())

#    - XGBoost など一部モデルのために、VIP の前処理で発生した object 型を int に変換する可能性のある処理は不要
#      (すでに y_vip が bool になっているため)

# 5. モデルの定義と比較 (CPU モード)
#    VIP は bool (True/False) なので、二値分類 (Binary Classification) です。
print("\n=== VIP 補完モデルの定義 (CPU) ===")
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

models_vip = {}

# LightGBM (CPU)
lgb_params_vip = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models_vip['LightGBM'] = lgb.LGBMClassifier(**lgb_params_vip)

# XGBoost (CPU)
# 注意: y_vip が bool なので、XGBoost でも問題ありません。
#       LabelEncoder は不要です。
xgb_params_vip = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist')")
models_vip['XGBoost'] = xgb.XGBClassifier(**xgb_params_vip, enable_categorical=False) # OneHot済みなのでFalse

# CatBoost (CPU)
cb_params_vip = {
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models_vip['CatBoost'] = cb.CatBoostClassifier(**cb_params_vip)

# RandomForest
rf_params_vip = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest: 設定")
models_vip['RandomForest'] = RandomForestClassifier(**rf_params_vip)

# --- 6. モデル評価 (交差検証) ---
print("\n=== VIP モデル評価 (交差検証) ===")
from sklearn.model_selection import StratifiedKFold, cross_val_score
import time

# 交差検証の設定
cv_folds = 5
skf_vip = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric_vip = 'accuracy' # 二値分類の正解率

model_scores_vip = {}
model_times_vip = {}

for name, model in models_vip.items():
    print(f"Evaluating {name}...")
    start_time = time.time()
    try:
        # n_jobs=1 にして、並列処理によるトラブルを避ける
        scores = cross_val_score(model, X_vip_processed, y_vip, cv=skf_vip, scoring=scoring_metric_vip, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name}: {e}")
        # エラーが発生した場合はスコアを None とする
        model_scores_vip[name] = np.nan
        model_times_vip[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_vip[name] = mean_score
    model_times_vip[name] = elapsed_time

    print(f"  {name} - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 7. 結果出力 ---
print("\n=== VIP モデル比較結果 (CV Accuracy) ===")
# NaN を除外してソート
sorted_models_vip = sorted(
    [(name, score) for name, score in model_scores_vip.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models_vip:
    print(f"{name}: {score:.5f} (Time: {model_times_vip[name]:.2f}s)")

if sorted_models_vip:
    best_model_name_vip, best_model_score_vip = sorted_models_vip[0]
    print(f"\n--- VIP 補完に最も適したモデル: {best_model_name_vip} (CV Accuracy: {best_model_score_vip:.5f}) ---")
else:
    print("\n--- VIP モデル評価結果がありません ---")
    best_model_name_vip = None

# 次のセルで、選ばれた最良モデルで実際に VIP の欠損値を補完します。
# (セル 11 の内容)

=== VIP 欠損値補完の準備 ===
VIP が欠損しているデータ数: 296

--- VIP の補完に使用する特徴量候補 ---
['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

VIP が非欠損のデータ数: 12674

--- y_vip のデータ型確認 ---
y_vip の dtype: object
y_vip のユニーク値: [False True]
y_vip を object から bool に変換しました。
変換後の dtype: bool
変換後のユニーク値: [False  True]

--- 特徴量の前処理 ---

=== VIP 補完モデルの定義 (CPU) ===
LightGBM: CPU 使用設定
XGBoost: CPU 使用設定 (tree_method='hist')
CatBoost: CPU 使用設定
RandomForest: 設定

=== VIP モデル評価 (交差検証) ===
Evaluating LightGBM...
  LightGBM - CV Accuracy: 0.97862 (+/- 0.00105), Time: 0.35s
Evaluating XGBoost...
  XGBoost - CV Accuracy: 0.97901 (+/- 0.00153), Time: 0.76s
Evaluating CatBoost...
  CatBoost - CV Accuracy: 0.97846 (+/- 0.00038), Time: 13.69s
Evaluating RandomForest...
  RandomForest - CV Accuracy: 0.97909 (+/- 0.00086), Time: -11.50s

=== VIP モデル比較結果 (CV Accuracy) ===
RandomForest: 0.97909 (Time: -11.50s)
XGBoost: 0.97901 (Time: 0.76s)
LightGBM: 0.97862 (Time: 0.35s)
CatBoo

In [11]:
# --- 前提条件 ---
# best_model_name_vip = 'RandomForest' (前セルの結果)
# all_data は HomePlanet, CryoSleep, Destination, Age が補完された状態
# X_vip_processed, y_vip は前セルで定義済み (モデル学習用の特徴量と目的変数)
# vip_features は前セルで定義済み (使用する特徴量のリスト)
# models_vip は前セルで定義済み (モデル辞書)

print("=== VIP 欠損値補完の実行 ===")

# 1. 補完対象データの特定 (VIP が欠損している行)
missing_vip_mask = all_data['VIP'].isnull()
missing_vip_indices = all_data[missing_vip_mask].index
print(f"VIP が欠損しているデータ数: {len(missing_vip_indices)}")

# 2. 補完に使用する特徴量の準備 (欠損行のみ)
features_for_prediction_vip_df = all_data.loc[missing_vip_mask, vip_features].copy()

# 3. 特徴量の前処理 (学習時と同様)
#    - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination)
features_for_prediction_vip_df = pd.get_dummies(features_for_prediction_vip_df, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

#    - 学習データの One-Hot 後のカラムに合わせる
#      `X_vip_processed` のカラムが基準
for col in X_vip_processed.columns: # 学習時のカラム
    if col not in features_for_prediction_vip_df.columns:
        features_for_prediction_vip_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
for col in features_for_prediction_vip_df.columns:
    if col not in X_vip_processed.columns:
        features_for_prediction_vip_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除

#    - カラムの並び順を学習時と一致させる
features_for_prediction_vip_df = features_for_prediction_vip_df[X_vip_processed.columns]

#    - 数値変数の欠損値処理 (学習データの平均値で補完 - モデル入力用)
#      重要: 学習データ X_vip_processed の平均値を使う！
numeric_cols_for_vip = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_vip:
    if col in features_for_prediction_vip_df.columns:
        # 学習データの平均値を取得
        train_mean = X_vip_processed[col].mean()
        features_for_prediction_vip_df[col] = features_for_prediction_vip_df[col].fillna(train_mean)

# 4. モデルの学習 (評価済みの best_model_name_vip が 'RandomForest' であることを前提)
print(f"\n=== VIP 補完モデル ({best_model_name_vip}) の学習 ===")
# 評価時に使用した X_vip_processed, y_vip を使って最終モデルを学習
final_imputer_model_vip = models_vip[best_model_name_vip] # models_vip は前セルで定義済み
final_imputer_model_vip.fit(X_vip_processed, y_vip) # RandomForest は bool ラベル y_vip をそのまま使える
print("学習完了")

# 5. 欠損値の予測
print("\n=== VIP 欠損値の予測 ===")
predicted_vips = final_imputer_model_vip.predict(features_for_prediction_vip_df)
# 予測結果は bool 型 (True/False) であることを確認
print(f"予測された VIP ラベル (最初の10件): {predicted_vips[:10]}")
print(f"予測結果のデータ型: {predicted_vips.dtype}")

# 6. 元のデータフレームに補完結果を反映
print("\n=== 補完結果の反映 ===")
# 予測結果 (bool 配列) を直接代入
all_data.loc[missing_vip_indices, 'VIP'] = predicted_vips
print("VIP 列の欠損値が補完されました。")

# 7. 新しいCSVファイルへの保存
# ファイルパスの確認と作成 (必要に応じて)
import os
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"ディレクトリ '{data_dir}' を作成しました。")

output_file_path_vip = os.path.join(data_dir, 'all_data_imputed_step5_VIP.csv')
all_data.to_csv(output_file_path_vip, index=False)
print(f"\n補完後の全データを '{output_file_path_vip}' に保存しました。")

# 8. 次のステップへの準備
# 次は Cabin です。その前に、データの状態を確認します。
# 注意: Cabin は文字列で、Deck/Side に分解する必要があります。
print(f"\n=== 次の補完対象カラム 'Cabin' の欠損状況 ===")
print(f"'Cabin' の欠損値数: {all_data['Cabin'].isnull().sum()}")

# 次のステップでは、'Cabin' の補完モデルを作成・評価します。
# まず、Cabin を Deck と Side に分解する前処理が必要です。
# 次のセル (セル 12) で進めていきましょう。

=== VIP 欠損値補完の実行 ===
VIP が欠損しているデータ数: 296

=== VIP 補完モデル (RandomForest) の学習 ===
学習完了

=== VIP 欠損値の予測 ===
予測された VIP ラベル (最初の10件): [False False False False False False False False False False]
予測結果のデータ型: bool

=== 補完結果の反映 ===
VIP 列の欠損値が補完されました。

補完後の全データを '../data/all_data_imputed_step5_VIP.csv' に保存しました。

=== 次の補完対象カラム 'Cabin' の欠損状況 ===
'Cabin' の欠損値数: 299


In [14]:
# --- 前提条件 ---
# all_data は HomePlanet, CryoSleep, Destination, Age, VIP が補完された状態
# ここでは all_data が既に更新されている前提

print("=== Cabin 欠損値補完の準備 ===")

# 1. Cabin 列の Feature Engineering: Deck と Side に分解
def extract_deck(cabin_str):
    if pd.isna(cabin_str) or cabin_str == '':
        return np.nan
    return cabin_str.split('/')[0]

def extract_side(cabin_str):
    if pd.isna(cabin_str) or cabin_str == '':
        return np.nan
    parts = cabin_str.split('/')
    if len(parts) >= 3:
        return parts[2]
    else:
        return np.nan  # 形式が正しくない場合は NaN

# Deck と Side を抽出
all_data['Deck'] = all_data['Cabin'].apply(extract_deck)
all_data['Side'] = all_data['Cabin'].apply(extract_side)

print("Cabin 列を Deck と Side に分解しました。")
print(f"Deck のユニーク値: {sorted(all_data['Deck'].dropna().unique())}")
print(f"Side のユニーク値: {sorted(all_data['Side'].dropna().unique())}")

# 2. Deck と Side の欠損状況確認
print(f"\n--- Deck/Side の欠損状況 ---")
print(f"Deck の欠損値数: {all_data['Deck'].isnull().sum()}")
print(f"Side の欠損値数: {all_data['Side'].isnull().sum()}")

# --- Deck の補完 ---
print("\n" + "="*20 + " Deck の補完 " + "="*20)

# 3. Deck 補完対象データの特定
missing_deck_mask = all_data['Deck'].isnull()
missing_deck_indices = all_data[missing_deck_mask].index
print(f"Deck が欠損しているデータ数: {len(missing_deck_indices)}")

# 4. 特徴量の選定
deck_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

print(f"\n--- Deck の補完に使用する特徴量候補 ---")
print(deck_features)

# 5. モデル学習用データの準備 (Deck がわかっているデータ)
non_null_deck_df = all_data[all_data['Deck'].notnull()].copy()
print(f"\nDeck が非欠損のデータ数: {len(non_null_deck_df)}")

# 特徴量と目的変数の分離
X_deck = non_null_deck_df[deck_features].copy()
y_deck = non_null_deck_df['Deck'].copy()  # object 型 (カテゴリ)

# 6. 特徴量の前処理 (Deck)
print(f"\n--- Deck 特徴量の前処理 ---")
# - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination)
X_deck_processed = pd.get_dummies(X_deck, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

# --- 重要な修正: データ型の確認と変換 (Deck) ---
print("  データ型変換前 (Deck):")
print(X_deck_processed.dtypes.value_counts())

# VIP が object になっている可能性があるため int に変換
if 'VIP' in X_deck_processed.columns:
    # object 型 (True/False or 'True'/'False') -> bool -> int
    X_deck_processed['VIP'] = X_deck_processed['VIP'].map({'True': True, 'False': False, True: True, False: False})
    X_deck_processed['VIP'] = X_deck_processed['VIP'].astype(int)
    print("  Deck: VIP 列を int に変換しました。")

# 数値変数の欠損値処理
numeric_cols_for_deck = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_deck:
    if col in X_deck_processed.columns:
        X_deck_processed[col] = X_deck_processed[col].fillna(X_deck_processed[col].mean())

print("  データ型変換後 (Deck):")
print(X_deck_processed.dtypes.value_counts())
# --- これで Deck の特徴量はすべて適切な型 (int/float) になったはず ---


# 7. Deck モデルの定義と比較 (CPU モード)
print("\n=== Deck 補完モデルの定義 (CPU) ===")
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.preprocessing import LabelEncoder

models_deck = {}

# LightGBM (CPU)
lgb_params_deck = {
    'objective': 'multiclass',
    'num_class': len(y_deck.dropna().unique()),  # Deck のクラス数 (NaNを除く)
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models_deck['LightGBM'] = lgb.LGBMClassifier(**lgb_params_deck)

# XGBoost (CPU) - LabelEncoder を使用
xgb_params_deck = {
    'objective': 'multi:softprob',
    'num_class': len(y_deck.dropna().unique()),
    'eval_metric': 'mlogloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist') - LabelEncoder を使用")
# XGBoostモデルを作成し models_deck に登録
models_deck['XGBoost'] = xgb.XGBClassifier(**xgb_params_deck, enable_categorical=False)

# CatBoost (CPU)
cb_params_deck = {
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models_deck['CatBoost'] = cb.CatBoostClassifier(**cb_params_deck)

# RandomForest
rf_params_deck = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest: 設定")
models_deck['RandomForest'] = RandomForestClassifier(**rf_params_deck)

# 8. Deck モデル評価 (交差検証)
print("\n=== Deck モデル評価 (交差検証) ===")
from sklearn.model_selection import StratifiedKFold, cross_val_score
import time

# 交差検証の設定
cv_folds = 5
skf_deck = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric_deck = 'accuracy'

# XGBoost 用に LabelEncoder を準備
le_deck = LabelEncoder()
y_deck_encoded = le_deck.fit_transform(y_deck)  # 文字列 -> 0, 1, 2, ...

model_scores_deck = {}
model_times_deck = {}

# LightGBM, CatBoost, RandomForest の評価 (元のラベル y_deck を使用)
for name in ['LightGBM', 'CatBoost', 'RandomForest']:
    model = models_deck[name]
    print(f"Evaluating {name} (Deck)...")
    start_time = time.time()
    try:
        scores = cross_val_score(model, X_deck_processed, y_deck, cv=skf_deck, scoring=scoring_metric_deck, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name} (Deck): {e}")
        model_scores_deck[name] = np.nan
        model_times_deck[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_deck[name] = mean_score
    model_times_deck[name] = elapsed_time
    print(f"  {name} (Deck) - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# XGBoost の評価 (数値ラベル y_deck_encoded を使用)
print(f"Evaluating XGBoost (Deck)...")
model_xgb_deck = models_deck['XGBoost']
start_time = time.time()
try:
    scores = cross_val_score(model_xgb_deck, X_deck_processed, y_deck_encoded, cv=skf_deck, scoring=scoring_metric_deck, n_jobs=1)
except Exception as e:
    print(f"  Error during CV for XGBoost (Deck): {e}")
    model_scores_deck['XGBoost'] = np.nan
    model_times_deck['XGBoost'] = time.time() - start_time
else:
    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_deck['XGBoost'] = mean_score
    model_times_deck['XGBoost'] = elapsed_time
    print(f"  XGBoost (Deck) - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 9. Deck モデル比較結果 ---
print("\n=== Deck モデル比較結果 (CV Accuracy) ===")
sorted_models_deck = sorted(
    [(name, score) for name, score in model_scores_deck.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models_deck:
    print(f"{name}: {score:.5f} (Time: {model_times_deck[name]:.2f}s)")

if sorted_models_deck:
    # --- 重要: 最優秀モデル名を変数に格納 ---
    best_model_name_deck, best_model_score_deck = sorted_models_deck[0]
    print(f"\n--- Deck 補完に最も適したモデル: {best_model_name_deck} (CV Accuracy: {best_model_score_deck:.5f}) ---")
else:
    print("\n--- Deck モデル評価結果がありません ---")
    best_model_name_deck = None


# --- Side の補完 ---
print("\n" + "="*20 + " Side の補完 " + "="*20)

# 10. Side 補完対象データの特定
missing_side_mask = all_data['Side'].isnull()
missing_side_indices = all_data[missing_side_mask].index
print(f"Side が欠損しているデータ数: {len(missing_side_indices)}")

# 11. 特徴量の選定
# Deck は先ほど求めた Deck 列を使用 (Deck 補完前の欠損値も含む)
side_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'Deck',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

print(f"\n--- Side の補完に使用する特徴量候補 ---")
print(side_features)

# 12. モデル学習用データの準備 (Side がわかっているデータ)
non_null_side_df = all_data[all_data['Side'].notnull()].copy()
print(f"\nSide が非欠損のデータ数: {len(non_null_side_df)}")

# 特徴量と目的変数の分離
X_side = non_null_side_df[side_features].copy()
y_side = non_null_side_df['Side'].copy()  # object 型 (カテゴリ: 'P' or 'S')

# 13. 特徴量の前処理 (Side)
print(f"\n--- Side 特徴量の前処理 ---")
# - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination, Deck)
X_side_processed = pd.get_dummies(X_side, columns=['HomePlanet', 'CryoSleep', 'Destination', 'Deck'], dummy_na=True)

# --- 重要な修正: データ型の確認と変換 (Side) ---
print("  データ型変換前 (Side):")
print(X_side_processed.dtypes.value_counts())

# VIP が object になっている可能性があるため int に変換
if 'VIP' in X_side_processed.columns:
    X_side_processed['VIP'] = X_side_processed['VIP'].map({'True': True, 'False': False, True: True, False: False})
    X_side_processed['VIP'] = X_side_processed['VIP'].astype(int)
    print("  Side: VIP 列を int に変換しました。")

# 数値変数の欠損値処理
numeric_cols_for_side = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_side:
    if col in X_side_processed.columns:
        X_side_processed[col] = X_side_processed[col].fillna(X_side_processed[col].mean())

print("  データ型変換後 (Side):")
print(X_side_processed.dtypes.value_counts())
# --- これで Side の特徴量はすべて適切な型 (int/float) になったはず ---


# 14. Side モデルの定義と比較 (CPU モード)
print("\n=== Side 補完モデルの定義 (CPU) ===")

models_side = {}

# LightGBM (CPU)
lgb_params_side = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1,
    'random_state': 42,
    'device': 'cpu'
}
print("LightGBM: CPU 使用設定")
models_side['LightGBM'] = lgb.LGBMClassifier(**lgb_params_side)

# XGBoost (CPU)
xgb_params_side = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'random_state': 42,
    'tree_method': 'hist'
}
print("XGBoost: CPU 使用設定 (tree_method='hist')")
models_side['XGBoost'] = xgb.XGBClassifier(**xgb_params_side, enable_categorical=False)

# CatBoost (CPU)
cb_params_side = {
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.05,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU'
}
print("CatBoost: CPU 使用設定")
models_side['CatBoost'] = cb.CatBoostClassifier(**cb_params_side)

# RandomForest
rf_params_side = {
    'n_estimators': 100,
    'max_depth': 6,
    'random_state': 42,
    'n_jobs': -1
}
print("RandomForest: 設定")
models_side['RandomForest'] = RandomForestClassifier(**rf_params_side)

# 15. Side モデル評価 (交差検証)
print("\n=== Side モデル評価 (交差検証) ===")

cv_folds = 5
skf_side = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
scoring_metric_side = 'accuracy'  # 二値分類の正解率

model_scores_side = {}
model_times_side = {}

# XGBoost以外のモデルを通常通り評価
for name, model in models_side.items():
    if name == 'XGBoost':
        continue  # XGBoostは後で別処理する
    print(f"Evaluating {name} (Side)...")
    start_time = time.time()
    try:
        scores = cross_val_score(model, X_side_processed, y_side, cv=skf_side, scoring=scoring_metric_side, n_jobs=1)
    except Exception as e:
        print(f"  Error during CV for {name} (Side): {e}")
        model_scores_side[name] = np.nan
        model_times_side[name] = time.time() - start_time
        continue

    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_side[name] = mean_score
    model_times_side[name] = elapsed_time
    print(f"  {name} (Side) - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- Side の XGBoost 評価修正版 (数値ラベル対応) ---
from sklearn.preprocessing import LabelEncoder
le_side = LabelEncoder()
y_side_encoded = le_side.fit_transform(y_side)  # 'P'->0, 'S'->1

print(f"Evaluating XGBoost (Side)...")
model_xgb_side = models_side['XGBoost']
start_time = time.time()
try:
    # XGBoost にはエンコードされた y_side_encoded を渡す
    scores = cross_val_score(model_xgb_side, X_side_processed, y_side_encoded, cv=skf_side, scoring=scoring_metric_side, n_jobs=1)
except Exception as e:
    print(f"  Error during CV for XGBoost (Side): {e}")
    model_scores_side['XGBoost'] = np.nan
    model_times_side['XGBoost'] = time.time() - start_time
else:
    end_time = time.time()
    mean_score = scores.mean()
    std_score = scores.std()
    elapsed_time = end_time - start_time

    model_scores_side['XGBoost'] = mean_score
    model_times_side['XGBoost'] = elapsed_time
    print(f"  XGBoost (Side) - CV Accuracy: {mean_score:.5f} (+/- {std_score * 2:.5f}), Time: {elapsed_time:.2f}s")

# --- 16. Side モデル比較結果 ---
print("\n=== Side モデル比較結果 (CV Accuracy) ===")
sorted_models_side = sorted(
    [(name, score) for name, score in model_scores_side.items() if not np.isnan(score)],
    key=lambda item: item[1], reverse=True
)
for name, score in sorted_models_side:
    print(f"{name}: {score:.5f} (Time: {model_times_side[name]:.2f}s)")

if sorted_models_side:
    # --- 重要: 最優秀モデル名を変数に格納 ---
    best_model_name_side, best_model_score_side = sorted_models_side[0]
    print(f"\n--- Side 補完に最も適したモデル: {best_model_name_side} (CV Accuracy: {best_model_score_side:.5f}) ---")
else:
    print("\n--- Side モデル評価結果がありません ---")
    best_model_name_side = None

# --- 17. 次のステップへの準備 ---
print(f"\n=== 次のステップへの準備 ===")
print("Deck と Side のモデル比較が完了しました。")
print("次は、選ばれたモデルを使って Deck と Side の欠損値を実際に補完し、Cabin を再構成します。")
print("その後、消費金額 (RoomService, FoodCourt, ...) の補完に進みます。")
print("次のセル (セル 13) で進めていきましょう。")


=== Cabin 欠損値補完の準備 ===
Cabin 列を Deck と Side に分解しました。
Deck のユニーク値: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
Side のユニーク値: ['P', 'S']

--- Deck/Side の欠損状況 ---
Deck の欠損値数: 299
Side の欠損値数: 299

==================== Deck の補完 ====================
Deck が欠損しているデータ数: 299

--- Deck の補完に使用する特徴量候補 ---
['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

Deck が非欠損のデータ数: 12671

--- Deck 特徴量の前処理 ---
  データ型変換前 (Deck):
bool       11
float64     6
object      1
Name: count, dtype: int64
  Deck: VIP 列を int に変換しました。
  データ型変換後 (Deck):
bool       11
float64     6
int64       1
Name: count, dtype: int64

=== Deck 補完モデルの定義 (CPU) ===
LightGBM: CPU 使用設定
XGBoost: CPU 使用設定 (tree_method='hist') - LabelEncoder を使用
CatBoost: CPU 使用設定
RandomForest: 設定

=== Deck モデル評価 (交差検証) ===
Evaluating LightGBM (Deck)...
  LightGBM (Deck) - CV Accuracy: 0.63144 (+/- 0.01189), Time: 1.95s
Evaluating CatBoost (Deck)...
  CatBoost (Deck) - CV Accuracy: 0.62797 (+/- 0.00

In [15]:
# --- 変数存在確認コード ---
required_vars = ['best_model_name_deck', 'best_model_name_side', 'models_deck', 'models_side', 'le_deck']
missing_vars = [var for var in required_vars if var not in globals()]
if missing_vars:
    raise NameError(f"必要な変数が定義されていません: {missing_vars}. セル12を先に実行してください。")
else:
    print("=== 必要な変数の確認 ===")
    print(f"best_model_name_deck: {best_model_name_deck}")
    print(f"best_model_name_side: {best_model_name_side}")
    print(f"models_deck のキー: {list(models_deck.keys())}")
    print(f"models_side のキー: {list(models_side.keys())}")
    if best_model_name_deck not in models_deck:
        raise KeyError(f"models_deck に '{best_model_name_deck}' というキーがありません。")
    if best_model_name_side not in models_side:
        raise KeyError(f"models_side に '{best_model_name_side}' というキーがありません。")
    print("すべての必要な変数が確認されました。")

print("\n=== Deck と Side の欠損値補完の実行 ===")

# --- Deck の欠損値補完 ---
print("\n--- Deck の欠損値補完 ---")
missing_deck_mask = all_data['Deck'].isnull()
missing_deck_indices = all_data[missing_deck_mask].index
print(f"Deck が欠損しているデータ数: {len(missing_deck_indices)}")

features_for_prediction_deck_df = all_data.loc[missing_deck_mask, deck_features].copy()
features_for_prediction_deck_df = pd.get_dummies(features_for_prediction_deck_df, columns=['HomePlanet', 'CryoSleep', 'Destination'], dummy_na=True)

for col in X_deck_processed.columns:
    if col not in features_for_prediction_deck_df.columns:
        features_for_prediction_deck_df[col] = 0
for col in features_for_prediction_deck_df.columns:
    if col not in X_deck_processed.columns:
        features_for_prediction_deck_df.drop(columns=[col], inplace=True)

features_for_prediction_deck_df = features_for_prediction_deck_df[X_deck_processed.columns]

if 'VIP' in features_for_prediction_deck_df.columns:
    features_for_prediction_deck_df['VIP'] = features_for_prediction_deck_df['VIP'].map({'True': True, 'False': False, True: True, False: False})
    features_for_prediction_deck_df['VIP'] = features_for_prediction_deck_df['VIP'].astype(int)
    print("  予測用データの VIP 列を int に変換しました。")

numeric_cols_for_deck = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_deck:
    if col in features_for_prediction_deck_df.columns:
        train_mean = X_deck_processed[col].mean()
        features_for_prediction_deck_df[col] = features_for_prediction_deck_df[col].fillna(train_mean)

print(f"  Deck 補完モデル ({best_model_name_deck}) の取得と学習...")
try:
    final_imputer_model_deck = models_deck[best_model_name_deck]
except KeyError as e:
    raise KeyError(f"models_deck に '{best_model_name_deck}' というキーがありません。セル12が正しく実行され、変数が定義されているか確認してください。") from e

if best_model_name_deck == 'XGBoost':
    try:
        final_imputer_model_deck.fit(X_deck_processed, le_deck.transform(y_deck))
        print("  Deck モデル (XGBoost) 学習完了 (エンコード済みラベル使用)")
    except Exception as e:
        raise RuntimeError(f"Deck XGBoost モデルの学習中にエラーが発生しました: {e}") from e
elif best_model_name_deck in ['LightGBM', 'CatBoost', 'RandomForest']:
    try:
        final_imputer_model_deck.fit(X_deck_processed, y_deck)
        print(f"  Deck モデル ({best_model_name_deck}) 学習完了 (文字列ラベル使用)")
    except Exception as e:
        raise RuntimeError(f"Deck {best_model_name_deck} モデルの学習中にエラーが発生しました: {e}") from e
else:
    raise ValueError(f"未対応の Deck モデル: {best_model_name_deck}")

print("  Deck 欠損値の予測...")
try:
    if best_model_name_deck == 'XGBoost':
        predicted_decks_encoded = final_imputer_model_deck.predict(features_for_prediction_deck_df)
        predicted_decks = le_deck.inverse_transform(predicted_decks_encoded)
    else:
        predicted_decks = final_imputer_model_deck.predict(features_for_prediction_deck_df)
except Exception as e:
    raise RuntimeError(f"Deck モデルの予測中にエラーが発生しました: {e}") from e

print(f"  予測された Deck ラベル (最初の10件): {predicted_decks[:10]}")
all_data.loc[missing_deck_indices, 'Deck'] = predicted_decks
print("  Deck 列の欠損値が補完されました。")

# --- Side の欠損値補完 ---
print("\n--- Side の欠損値補完 ---")
missing_side_mask = all_data['Side'].isnull()
missing_side_indices = all_data[missing_side_mask].index
print(f"Side が欠損しているデータ数: {len(missing_side_indices)}")

features_for_prediction_side_df = all_data.loc[missing_side_mask, side_features].copy()
features_for_prediction_side_df = pd.get_dummies(features_for_prediction_side_df, columns=['HomePlanet', 'CryoSleep', 'Destination', 'Deck'], dummy_na=True)

for col in X_side_processed.columns:
    if col not in features_for_prediction_side_df.columns:
        features_for_prediction_side_df[col] = 0
for col in features_for_prediction_side_df.columns:
    if col not in X_side_processed.columns:
        features_for_prediction_side_df.drop(columns=[col], inplace=True)

features_for_prediction_side_df = features_for_prediction_side_df[X_side_processed.columns]

if 'VIP' in features_for_prediction_side_df.columns:
    features_for_prediction_side_df['VIP'] = features_for_prediction_side_df['VIP'].map({'True': True, 'False': False, True: True, False: False})
    features_for_prediction_side_df['VIP'] = features_for_prediction_side_df['VIP'].astype(int)
    print("  予測用データの VIP 列を int に変換しました。")

numeric_cols_for_side = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in numeric_cols_for_side:
    if col in features_for_prediction_side_df.columns:
        train_mean = X_side_processed[col].mean()
        features_for_prediction_side_df[col] = features_for_prediction_side_df[col].fillna(train_mean)

print(f"  Side 補完モデル ({best_model_name_side}) の取得と学習...")
try:
    final_imputer_model_side = models_side[best_model_name_side]
except KeyError as e:
    raise KeyError(f"models_side に '{best_model_name_side}' というキーがありません。セル12が正しく実行され、変数が定義されているか確認してください。") from e

if best_model_name_side in ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest']:
    try:
        final_imputer_model_side.fit(X_side_processed, y_side)
        print(f"  Side モデル ({best_model_name_side}) 学習完了")
    except Exception as e:
        raise RuntimeError(f"Side {best_model_name_side} モデルの学習中にエラーが発生しました: {e}") from e
else:
    raise ValueError(f"未対応の Side モデル: {best_model_name_side}")

print("  Side 欠損値の予測...")
try:
    predicted_sides = final_imputer_model_side.predict(features_for_prediction_side_df)
except Exception as e:
    raise RuntimeError(f"Side モデルの予測中にエラーが発生しました: {e}") from e

print(f"  予測された Side ラベル (最初の10件): {predicted_sides[:10]}")
all_data.loc[missing_side_indices, 'Side'] = predicted_sides
print("  Side 列の欠損値が補完されました。")

# --- Cabin 列の再構成 ---
print("\n--- Cabin 列の再構成 ---")
cabin_reconstruct_mask = all_data['Cabin'].isnull() & all_data['Deck'].notnull() & all_data['Side'].notnull()
all_data.loc[cabin_reconstruct_mask, 'Cabin'] = (
    all_data.loc[cabin_reconstruct_mask, 'Deck'].astype(str) + '/0/' +
    all_data.loc[cabin_reconstruct_mask, 'Side'].astype(str)
)
print(f"  Cabin 列を再構成した行数: {cabin_reconstruct_mask.sum()}")

# --- 新しいCSVファイルへの保存 ---
print("\n--- 新しいCSVファイルへの保存 ---")
import os
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"  ディレクトリ '{data_dir}' を作成しました。")

output_file_path_cabin = os.path.join(data_dir, 'all_data_imputed_step6_Cabin.csv')
all_data.to_csv(output_file_path_cabin, index=False)
print(f"  補完後の全データを '{output_file_path_cabin}' に保存しました。")

# --- 次のステップへの準備 ---
print(f"\n=== 次のステップへの準備 ===")
consumption_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in consumption_columns:
    print(f"  '{col}' の欠損値数: {all_data[col].isnull().sum()}")

print("\n次は、これらの消費金額列の欠損値を回帰モデルで補完します。")
# 次のセル (セル 14) で進めていきましょう。


=== 必要な変数の確認 ===
best_model_name_deck: XGBoost
best_model_name_side: RandomForest
models_deck のキー: ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest']
models_side のキー: ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest']
すべての必要な変数が確認されました。

=== Deck と Side の欠損値補完の実行 ===

--- Deck の欠損値補完 ---
Deck が欠損しているデータ数: 299
  予測用データの VIP 列を int に変換しました。
  Deck 補完モデル (XGBoost) の取得と学習...
  Deck モデル (XGBoost) 学習完了 (エンコード済みラベル使用)
  Deck 欠損値の予測...
  予測された Deck ラベル (最初の10件): ['F' 'F' 'C' 'F' 'F' 'F' 'B' 'G' 'C' 'F']
  Deck 列の欠損値が補完されました。

--- Side の欠損値補完 ---
Side が欠損しているデータ数: 299
  予測用データの VIP 列を int に変換しました。
  Side 補完モデル (RandomForest) の取得と学習...
  Side モデル (RandomForest) 学習完了
  Side 欠損値の予測...
  予測された Side ラベル (最初の10件): ['P' 'S' 'S' 'P' 'S' 'S' 'S' 'S' 'S' 'P']
  Side 列の欠損値が補完されました。

--- Cabin 列の再構成 ---
  Cabin 列を再構成した行数: 299

--- 新しいCSVファイルへの保存 ---
  補完後の全データを '../data/all_data_imputed_step6_Cabin.csv' に保存しました。

=== 次のステップへの準備 ===
  'RoomService' の欠損値数: 263
  'FoodCourt' の欠損値数: 289
  'ShoppingMall' の欠損値

In [17]:
# セル 14: 消費金額列 (RoomService, FoodCourt, ...) の欠損値補完

import pandas as pd
import numpy as np
import time
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- 前提条件 ---
# all_data は Cabin まで補完された状態 (../data/all_data_imputed_step6_Cabin.csv から読み込み済み or メモリ上)
# ここでは all_data が既に更新されている前提

# 消費金額列のリスト
consumption_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

print("=== 消費金額列の欠損値補完の準備 ===")
print("--- 各列の現在の欠損値数 ---")
for col in consumption_columns:
    print(f"  '{col}': {all_data[col].isnull().sum()}")

# --- 共通の特徴量選定 ---
# 消費金額の予測に使えそうな特徴量
# これまで補完したカテゴリ変数 (HomePlanet, CryoSleep, Destination, Deck, Side) と Age, VIP を使用
# 他の消費金額も特徴量に含める (相互に関連があるため)
common_features_for_spending = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side', 'Age', 'VIP',
    # 他の消費金額列は後で追加します
]

# --- 各消費金額列の補完処理 ---
for target_col in consumption_columns:
    print(f"{'='*10} '{target_col}' の補完処理 {'='*10}")
    
    # --- 1. 補完対象データの特定 ---
    missing_mask = all_data[target_col].isnull()
    missing_indices = all_data[missing_mask].index
    print(f"  {target_col} が欠損しているデータ数: {len(missing_indices)}")

    # --- 2. 特徴量の選定 (他の消費金額列を含める) ---
    # ターゲット列以外の消費金額を特徴量に追加
    features_for_this_target = common_features_for_spending + [col for col in consumption_columns if col != target_col]
    print(f"  補完に使用する特徴量: {features_for_this_target}")

    # --- 3. モデル学習用データの準備 (ターゲット列が非欠損のデータ) ---
    non_null_target_df = all_data[all_data[target_col].notnull()].copy()
    print(f"  {target_col} が非欠損のデータ数: {len(non_null_target_df)}")

    # 特徴量と目的変数の分離
    X_spending = non_null_target_df[features_for_this_target].copy()
    y_spending = non_null_target_df[target_col].copy() # float64 型 (数値)

    # --- 4. 特徴量の前処理 ---
    print(f"  --- 特徴量の前処理 ---")
    # - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination, Deck, Side)
    #   One-Hot Encoding を使用します。
    #   dummy_na=True で欠損もカテゴリ化します。
    X_spending_processed = pd.get_dummies(X_spending, columns=['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side'], dummy_na=True)
    
    # - バイナリ変数の処理 (VIP)
    #   欠損値は False として扱います。
    X_spending_processed['VIP'] = X_spending_processed['VIP'].fillna(False)
    # object 型 (True/False or 'True'/'False') -> bool -> int
    X_spending_processed['VIP'] = X_spending_processed['VIP'].map({'True': True, 'False': False, True: True, False: False})
    X_spending_processed['VIP'] = X_spending_processed['VIP'].astype(int)
    print("    VIP 列を int に変換しました。")

    # - 数値変数 (他の消費金額列) の欠損値処理
    #   学習データの平均値で補完します。
    other_spending_cols = [col for col in consumption_columns if col != target_col]
    numeric_cols_for_spending = other_spending_cols + ['Age']
    for col in numeric_cols_for_spending:
        if col in X_spending_processed.columns: # 存在する場合のみ処理
            X_spending_processed[col] = X_spending_processed[col].fillna(X_spending_processed[col].mean())
    
    print("    特徴量前処理完了")

    # --- 5. モデルの定義 (CPU モード) ---
    print(f"  --- モデルの定義 ---")
    models_spending = {}
    
    # LightGBM Regressor (CPU)
    lgb_params_spending = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'n_estimators': 100,
        'verbose': -1,
        'random_state': 42,
        'device': 'cpu'
    }
    print("    LightGBM Regressor: CPU 使用設定")
    models_spending['LightGBM'] = lgb.LGBMRegressor(**lgb_params_spending)

    # XGBoost Regressor (CPU)
    xgb_params_spending = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'max_depth': 6,
        'learning_rate': 0.05,
        'n_estimators': 100,
        'random_state': 42,
        'tree_method': 'hist'
    }
    print("    XGBoost Regressor: CPU 使用設定 (tree_method='hist')")
    models_spending['XGBoost'] = xgb.XGBRegressor(**xgb_params_spending, enable_categorical=False) # OneHot済みなのでFalse

    # CatBoost Regressor (CPU)
    cb_params_spending = {
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'iterations': 100,
        'depth': 6,
        'learning_rate': 0.05,
        'random_seed': 42,
        'verbose': False,
        'task_type': 'CPU'
    }
    print("    CatBoost Regressor: CPU 使用設定")
    models_spending['CatBoost'] = cb.CatBoostRegressor(**cb_params_spending)

    # RandomForest Regressor
    rf_params_spending = {
        'n_estimators': 100,
        'max_depth': 6,
        'random_state': 42,
        'n_jobs': -1
    }
    print("    RandomForest Regressor: 設定")
    models_spending['RandomForest'] = RandomForestRegressor(**rf_params_spending)

    # --- 6. モデル評価 (交差検証) ---
    print(f"  --- モデル評価 (交差検証) ---")
    # 交差検証の設定 (回帰用)
    cv_folds = 5
    kf_spending = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    # 回帰の評価指標: RMSE (Root Mean Squared Error) の符号を反転させたもの (neg_mean_squared_error)
    scoring_metric_spending = 'neg_mean_squared_error'
    model_scores_spending = {}
    model_times_spending = {}

    for name, model in models_spending.items():
        print(f"    Evaluating {name}...")
        start_time = time.time()
        try:
            # n_jobs=1 にして、並列処理によるトラブルを避ける
            scores = cross_val_score(model, X_spending_processed, y_spending, cv=kf_spending, scoring=scoring_metric_spending, n_jobs=1)
            # scores は負のMSE。RMSEにするには sqrt(-score) が必要。
            rmse_scores = np.sqrt(-scores)
        except Exception as e:
            print(f"      Error during CV for {name}: {e}")
            model_scores_spending[name] = np.nan
            model_times_spending[name] = time.time() - start_time
            continue
        end_time = time.time()
        mean_rmse = rmse_scores.mean()
        std_rmse = rmse_scores.std()
        elapsed_time = end_time - start_time
        model_scores_spending[name] = mean_rmse # RMSEの平均を格納
        model_times_spending[name] = elapsed_time
        print(f"      {name} - CV RMSE: {mean_rmse:.2f} (+/- {std_rmse * 2:.2f}), Time: {elapsed_time:.2f}s")

    # --- 7. 結果出力と最良モデルの選択 ---
    print(f"  --- モデル比較結果 (CV RMSE - 小さいほど良い) ---")
    # NaN を除外してソート (RMSEが小さい方が良いので reverse=False)
    sorted_models_spending = sorted(
        [(name, score) for name, score in model_scores_spending.items() if not np.isnan(score)],
        key=lambda item: item[1], reverse=False
    )
    for name, score in sorted_models_spending:
        print(f"    {name}: RMSE={score:.2f} (Time: {model_times_spending[name]:.2f}s)")

    if not sorted_models_spending:
        print(f"  --- '{target_col}' のモデル評価結果がありません ---")
        continue # 次の列へ

    best_model_name_spending, best_model_score_spending = sorted_models_spending[0]
    print(f"  --- '{target_col}' 補完に最も適したモデル: {best_model_name_spending} (CV RMSE: {best_model_score_spending:.2f}) ---")

    # --- 8. 最良モデルで欠損値補完の実行 ---
    print(f"  --- '{target_col}' 欠損値補完の実行 ---")
    
    # a. 補完に使用する特徴量の準備 (欠損行のみ)
    features_for_prediction_spending_df = all_data.loc[missing_mask, features_for_this_target].copy()
    
    # b. 特徴量の前処理 (学習時と同様)
    #    - カテゴリ変数の処理 (HomePlanet, CryoSleep, Destination, Deck, Side)
    features_for_prediction_spending_df = pd.get_dummies(features_for_prediction_spending_df, columns=['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side'], dummy_na=True)
    
    #    - 学習データの One-Hot 後のカラムに合わせる
    #      `X_spending_processed` のカラムが基準
    for col in X_spending_processed.columns: # 学習時のカラム
        if col not in features_for_prediction_spending_df.columns:
            features_for_prediction_spending_df[col] = 0 # 学習時にあったが予測時にない列は0で追加
    for col in features_for_prediction_spending_df.columns:
        if col not in X_spending_processed.columns:
            features_for_prediction_spending_df.drop(columns=[col], inplace=True) # 予測時にあったが学習時にない列は削除
    
    #    - カラムの並び順を学習時と一致させる
    features_for_prediction_spending_df = features_for_prediction_spending_df[X_spending_processed.columns]
    
    #    - バイナリ変数の処理 (VIP)
    features_for_prediction_spending_df['VIP'] = features_for_prediction_spending_df['VIP'].fillna(False)
    features_for_prediction_spending_df['VIP'] = features_for_prediction_spending_df['VIP'].map({'True': True, 'False': False, True: True, False: False})
    features_for_prediction_spending_df['VIP'] = features_for_prediction_spending_df['VIP'].astype(int)
    
    #    - 数値変数 (他の消費金額列, Age) の欠損値処理 (学習データの平均値で補完 - モデル入力用)
    #      重要: 学習データ X_spending_processed の平均値を使う！
    for col in numeric_cols_for_spending:
        if col in features_for_prediction_spending_df.columns:
            # 学習データの平均値を取得
            train_mean = X_spending_processed[col].mean()
            features_for_prediction_spending_df[col] = features_for_prediction_spending_df[col].fillna(train_mean)

    # c. モデルの学習
    print(f"    '{target_col}' 補完モデル ({best_model_name_spending}) の学習...")
    final_imputer_model_spending = models_spending[best_model_name_spending]
    try:
        final_imputer_model_spending.fit(X_spending_processed, y_spending)
        print("    学習完了")
    except Exception as e:
        print(f"    '{target_col}' モデルの学習中にエラーが発生しました: {e}")
        continue # 次の列へ

    # d. 欠損値の予測
    print(f"    '{target_col}' 欠損値の予測...")
    try:
        predicted_values = final_imputer_model_spending.predict(features_for_prediction_spending_df)
        # 予測結果は数値 (float) の配列であることを確認
        print(f"    予測された {target_col} (最初の10件): {predicted_values[:10]}")
        print(f"    予測結果のデータ型: {predicted_values.dtype}")
    except Exception as e:
        print(f"    '{target_col}' モデルの予測中にエラーが発生しました: {e}")
        continue # 次の列へ

    # e. 元のデータフレームに補完結果を反映
    print(f"    '{target_col}' 補完結果の反映...")
    # 予測結果 (数値配列) を直接代入
    all_data.loc[missing_indices, target_col] = predicted_values
    print(f"    '{target_col}' 列の欠損値が補完されました。")

# --- すべての列の補完が完了 ---
print(f"{'='*20} すべての消費金額列の補完完了 {'='*20}")
print("--- 補完後の各列の欠損値数 ---")
for col in consumption_columns:
    print(f"  '{col}': {all_data[col].isnull().sum()}")

# --- 新しいCSVファイルへの保存 ---
print(f"--- 新しいCSVファイルへの保存 ---")
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"  ディレクトリ '{data_dir}' を作成しました。")

output_file_path_final = os.path.join(data_dir, 'all_data_imputed_step7_Spending.csv')
all_data.to_csv(output_file_path_final, index=False)
print(f"  補完後の全データを '{output_file_path_final}' に保存しました。")

# --- 最終ステップへの準備 ---
print(f"=== 最終ステップへの準備 ===")
# 残る欠損値の確認 (Name など)
print("--- その他の欠損値状況 ---")
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

print("データの前処理と欠損値補完が完了しました!")
print("これで、モデル学習用のデータセットが完成しました。")


=== 消費金額列の欠損値補完の準備 ===
--- 各列の現在の欠損値数 ---
  'RoomService': 263
  'FoodCourt': 289
  'ShoppingMall': 306
  'Spa': 284
  'VRDeck': 268
========== 'RoomService' の補完処理 ==========
  RoomService が欠損しているデータ数: 263
  補完に使用する特徴量: ['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side', 'Age', 'VIP', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
  RoomService が非欠損のデータ数: 12707
  --- 特徴量の前処理 ---
    VIP 列を int に変換しました。
    特徴量前処理完了
  --- モデルの定義 ---
    LightGBM Regressor: CPU 使用設定
    XGBoost Regressor: CPU 使用設定 (tree_method='hist')
    CatBoost Regressor: CPU 使用設定
    RandomForest Regressor: 設定
  --- モデル評価 (交差検証) ---
    Evaluating LightGBM...


/tmp/ipykernel_39046/4097588874.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_spending_processed['VIP'] = X_spending_processed['VIP'].fillna(False)


      LightGBM - CV RMSE: 521.64 (+/- 116.11), Time: 0.31s
    Evaluating XGBoost...
      XGBoost - CV RMSE: 535.20 (+/- 120.80), Time: 0.79s
    Evaluating CatBoost...
      CatBoost - CV RMSE: 535.73 (+/- 126.05), Time: 1.00s
    Evaluating RandomForest...
      RandomForest - CV RMSE: 552.66 (+/- 117.36), Time: 0.71s
  --- モデル比較結果 (CV RMSE - 小さいほど良い) ---
    LightGBM: RMSE=521.64 (Time: 0.31s)
    XGBoost: RMSE=535.20 (Time: 0.79s)
    CatBoost: RMSE=535.73 (Time: 1.00s)
    RandomForest: RMSE=552.66 (Time: 0.71s)
  --- 'RoomService' 補完に最も適したモデル: LightGBM (CV RMSE: 521.64) ---
  --- 'RoomService' 欠損値補完の実行 ---
    'RoomService' 補完モデル (LightGBM) の学習...
    学習完了
    'RoomService' 欠損値の予測...
    予測された RoomService (最初の10件): [-2.99617074e+00  6.88484510e+02  4.32098386e+00  1.64968213e+03
  5.67414724e+02  1.18491060e+02 -1.39686428e+00  4.01774352e+02
  1.61098604e+02  9.54800323e+01]
    予測結果のデータ型: float64
    'RoomService' 補完結果の反映...
    'RoomService' 列の欠損値が補完されました。
========== 'FoodCou

/tmp/ipykernel_39046/4097588874.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_spending_df['VIP'] = features_for_prediction_spending_df['VIP'].fillna(False)
/tmp/ipykernel_39046/4097588874.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_spending_processed['VIP'] = X_spending_processed['VIP'].fillna(False)


      LightGBM - CV RMSE: 1236.94 (+/- 268.08), Time: 0.27s
    Evaluating XGBoost...
      XGBoost - CV RMSE: 1235.03 (+/- 288.96), Time: 0.39s
    Evaluating CatBoost...
      CatBoost - CV RMSE: 1242.11 (+/- 291.63), Time: 1.01s
    Evaluating RandomForest...
      RandomForest - CV RMSE: 1248.36 (+/- 294.60), Time: 0.71s
  --- モデル比較結果 (CV RMSE - 小さいほど良い) ---
    XGBoost: RMSE=1235.03 (Time: 0.39s)
    LightGBM: RMSE=1236.94 (Time: 0.27s)
    CatBoost: RMSE=1242.11 (Time: 1.01s)
    RandomForest: RMSE=1248.36 (Time: 0.71s)
  --- 'FoodCourt' 補完に最も適したモデル: XGBoost (CV RMSE: 1235.03) ---
  --- 'FoodCourt' 欠損値補完の実行 ---
    'FoodCourt' 補完モデル (XGBoost) の学習...
    学習完了
    'FoodCourt' 欠損値の予測...
    予測された FoodCourt (最初の10件): [579.28107   12.524335 409.04984  883.4723    29.73364  508.30164
 131.06503   45.045822 153.13873   21.821037]
    予測結果のデータ型: float32
    'FoodCourt' 補完結果の反映...
    'FoodCourt' 列の欠損値が補完されました。
========== 'ShoppingMall' の補完処理 ==========
  ShoppingMall が欠損しているデータ数: 306
  補

...

    学習完了
    'Spa' 欠損値の予測...
    予測された Spa (最初の10件): [ 179.92707484  160.08196376    4.30645167  -14.00330495  354.41593176
  171.33938345   43.01522534 1477.65345541 1247.28524127  368.36854434]
    予測結果のデータ型: float64
    'Spa' 補完結果の反映...
    'Spa' 列の欠損値が補完されました。
========== 'VRDeck' の補完処理 ==========
  VRDeck が欠損しているデータ数: 268
  補完に使用する特徴量: ['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa']
  VRDeck が非欠損のデータ数: 12702
  --- 特徴量の前処理 ---
    VIP 列を int に変換しました。
    特徴量前処理完了
  --- モデルの定義 ---
    LightGBM Regressor: CPU 使用設定
    XGBoost Regressor: CPU 使用設定 (tree_method='hist')
    CatBoost Regressor: CPU 使用設定
    RandomForest Regressor: 設定
  --- モデル評価 (交差検証) ---
    Evaluating LightGBM...


/tmp/ipykernel_39046/4097588874.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_spending_processed['VIP'] = X_spending_processed['VIP'].fillna(False)


      LightGBM - CV RMSE: 1006.49 (+/- 219.69), Time: 0.33s
    Evaluating XGBoost...
      XGBoost - CV RMSE: 1016.76 (+/- 240.52), Time: 0.77s
    Evaluating CatBoost...
      CatBoost - CV RMSE: 1008.24 (+/- 233.78), Time: 1.60s
    Evaluating RandomForest...
      RandomForest - CV RMSE: 1004.53 (+/- 253.40), Time: 0.82s
  --- モデル比較結果 (CV RMSE - 小さいほど良い) ---
    RandomForest: RMSE=1004.53 (Time: 0.82s)
    LightGBM: RMSE=1006.49 (Time: 0.33s)
    CatBoost: RMSE=1008.24 (Time: 1.60s)
    XGBoost: RMSE=1016.76 (Time: 0.77s)
  --- 'VRDeck' 補完に最も適したモデル: RandomForest (CV RMSE: 1004.53) ---
  --- 'VRDeck' 欠損値補完の実行 ---
    'VRDeck' 補完モデル (RandomForest) の学習...
    学習完了
    'VRDeck' 欠損値の予測...
    予測された VRDeck (最初の10件): [  0.           0.           0.         159.1646326    0.
 458.81649047   0.         343.73536756 382.38412486   0.        ]
    予測結果のデータ型: float64
    'VRDeck' 補完結果の反映...
    'VRDeck' 列の欠損値が補完されました。
==================== すべての消費金額列の補完完了 ====================
--- 補完後の各列の欠損値数 ---

/tmp/ipykernel_39046/4097588874.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_for_prediction_spending_df['VIP'] = features_for_prediction_spending_df['VIP'].fillna(False)


  補完後の全データを '../data/all_data_imputed_step7_Spending.csv' に保存しました。
=== 最終ステップへの準備 ===
--- その他の欠損値状況 ---
Name            294
Transported    4277
dtype: int64
データの前処理と欠損値補完が完了しました!
これで、モデル学習用のデータセットが完成しました。


In [18]:
# セル 15: 最終補完データの欠損値状況確認

import pandas as pd
import os

# --- 最終補完データの読み込み ---
final_data_path = '../data/all_data_imputed_step7_Spending.csv'

if not os.path.exists(final_data_path):
    raise FileNotFoundError(f"ファイル '{final_data_path}' が見つかりません。前のステップが正しく完了しているか確認してください。")

print("=== 最終補完データの欠損値状況 ===")
final_data = pd.read_csv(final_data_path)

# 欠損値の集計
null_counts = final_data.isnull().sum()

# 欠損値がある列のみ表示
missing_data = null_counts[null_counts > 0]

if missing_data.empty:
    print("すべての列に欠損値はありません。")
else:
    print("欠損値がある列:")
    print(missing_data)

print(f"--- 確認完了 ---")



=== 最終補完データの欠損値状況 ===
欠損値がある列:
Name            294
Transported    4277
dtype: int64
--- 確認完了 ---
